# 1. Configuration

In [ ]:
import os
import time
import sys
import json
from tqdm import tqdm
from datetime import datetime
from typing import Dict, List, Optional, Union, Tuple

# Data processing imports
import numpy as np
import pandas as pd

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning imports
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import FunctionTransformer

# Optional imports - handle gracefully if not available
try:
    from scipy import stats
    SCIPY_AVAILABLE = True
except ImportError:
    SCIPY_AVAILABLE = False

# Add parent directory to path for importing utilities
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir) if current_dir.endswith('notebooks') else current_dir
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import utility functions from the project
try:
    from src.utils.data_loader import (
        load_main_dataset, 
        load_trade_data, 
        load_data,
        get_token_ids_for_market,
        find_token_id_file
    )
except ImportError:
    print("Warning: Could not import data_loader utilities. Some functions may not work.")

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')



In [ ]:
# 1. Configuration
MARKET_SELECTION = {
    'by_name': [
        "Will Donald Trump win the 2024 US Presidential Election?",
        "Will Kamala Harris win the 2024 US Presidential Election?"
    ],
    'by_id': [],  # Add specific market IDs here if needed
    'top_n_by_volume': 5,  # Set to a number > 0 to analyze top N markets by volume
    'min_volume': 0,  # Minimum volume threshold
    'date_range': None,  # Set to (start_date, end_date) to filter by date
}
# List of known protocol/exchange IDs to exclude from trader analysis
PROTOCOL_ACCOUNTS = [
    "0xc5d563a36ae78145c45a50134d48a1215220f80a"  # NegRiskCtfExchange
]
# Analysis Configuration
ANALYSIS_CONFIG = {
    'run_trader_distribution': True,
    'run_whale_identification': True,
    'run_trader_classification': True,
    'run_market_dynamics': True,
    'whale_threshold': 0.01,  # Top 1% traders by volume are considered whales
    'trader_clusters': 5,  # Number of trader clusters for classification
    'save_results': True,  # Whether to save results to files
    'results_dir': 'results/trader_analysis',
    'generate_plots': True  # Whether to generate plots
}

# 2. Data Loading

In [ ]:
# More efficient version of load_trade_data_for_analysis
def load_trade_data_for_analysis(market_ids, cache_dir=None):
    """
    Load and combine trade data with caching option
    """
    all_trades = []
    successful_markets = 0
    cache_file = None
    
    if cache_dir:
        os.makedirs(cache_dir, exist_ok=True)
        cache_id = "_".join(str(m_id) for m_id in market_ids[:3])
        if len(market_ids) > 3:
            cache_id += f"_plus_{len(market_ids)-3}_more"
        cache_file = os.path.join(cache_dir, f"combined_trades_{cache_id}.parquet")
        
        # Check if cache exists
        if os.path.exists(cache_file):
            print(f"Loading cached trade data from {cache_file}")
            return pd.read_parquet(cache_file)
    
    # Original loading logic...
    for market_id in market_ids:
        print(f"Loading trade data for market {market_id}...")
        
        try:
            # Use the imported load_trade_data function
            market_trades = load_trade_data(market_id)
            
            if market_trades is not None and len(market_trades) > 0:
                # Add market_id if not already present
                if 'market_id' not in market_trades.columns:
                    market_trades['market_id'] = market_id
                
                all_trades.append(market_trades)
                successful_markets += 1
        except Exception as e:
            print(f"Error loading trade data for market {market_id}: {e}")
    
    if not all_trades:
        print("No trade data found for selected markets")
        return None
    
    combined_trades = pd.concat(all_trades, ignore_index=True)
    print(f"Loaded {len(combined_trades):,} trades from {successful_markets} markets")
    
    
    # Cache the result
    if cache_dir and cache_file and len(all_trades) > 0:
        combined_trades = pd.concat(all_trades, ignore_index=True)
        combined_trades.to_parquet(cache_file)
        print(f"Cached combined trades to {cache_file}")
        return combined_trades
        
    # Return original result if no caching
    return pd.concat(all_trades, ignore_index=True) if all_trades else None

In [ ]:
def load_market_data(market_config):
    """
    Load market data based on configuration
    
    Parameters:
    -----------
    market_config : dict
        Dictionary with market selection parameters
        
    Returns:
    --------
    tuple
        (market_data, trade_data) - DataFrames with market and trade data
    """
    print("Loading main dataset...")
    market_data = load_main_dataset('data/cleaned_election_data.csv')
    
    if market_data is None:
        print("Failed to load market data")
        return None, None
    
    # Filter markets based on configuration
    selected_markets = market_data.copy()
    
    # Filter by name if specified
    if market_config['by_name'] and len(market_config['by_name']) > 0:
        selected_markets = selected_markets[selected_markets['question'].isin(market_config['by_name'])]
        print(f"Selected {len(selected_markets)} markets by name")
    
    # Filter by ID if specified
    if market_config['by_id'] and len(market_config['by_id']) > 0:
        id_filter = selected_markets['id'].isin(market_config['by_id'])
        if len(selected_markets) > 0:
            selected_markets = selected_markets[id_filter]
        else:
            selected_markets = market_data[id_filter]
        print(f"Selected {len(selected_markets)} markets by ID")
    
    # Filter by top N by volume
    if market_config['top_n_by_volume'] > 0:
        if 'volumeNum' in market_data.columns:
            top_markets = market_data.sort_values('volumeNum', ascending=False).head(
                market_config['top_n_by_volume'])
            
            if len(selected_markets) > 0:
                # Intersect with already selected markets
                selected_markets = selected_markets[selected_markets['id'].isin(top_markets['id'])]
            else:
                selected_markets = top_markets
                
            print(f"Selected {len(selected_markets)} top markets by volume")
    
    # Apply minimum volume filter if specified
    if market_config['min_volume'] > 0 and 'volumeNum' in market_data.columns:
        volume_filter = selected_markets['volumeNum'] >= market_config['min_volume']
        selected_markets = selected_markets[volume_filter]
        print(f"Selected {len(selected_markets)} markets with minimum volume {market_config['min_volume']}")
    
    # If no markets were selected, use default selection
    if len(selected_markets) == 0:
        print("No markets matched selection criteria. Using default selection.")
        if market_config['top_n_by_volume'] > 0:
            selected_markets = market_data.sort_values('volumeNum', ascending=False).head(
                market_config['top_n_by_volume'])
        else:
            selected_markets = market_data.head(2)  # Default to first 2 markets
    
    print(f"Final selection: {len(selected_markets)} markets")
    
    # Display selected markets
    if len(selected_markets) > 0:
        print("\nSelected Markets:")
        for i, (idx, row) in enumerate(selected_markets.iterrows()):
            market_name = row['question'] if 'question' in row else f"Market {row['id']}"
            print(f"{i+1}. {market_name} (ID: {row['id']})")
    
    # Load trade data for selected markets
    market_ids = selected_markets['id'].tolist()
    trade_data = load_trade_data_for_analysis(market_ids=market_ids)
    
    return selected_markets, trade_data

# 3. Data Processing

In [ ]:
# Optimize preprocess_trade_data with better documentation and progress tracking
def preprocess_trade_data(trade_data, sample_size=None, exclude_protocols=True):
    """
    Preprocess trade data with option to use a sample for faster development/testing
    
    Parameters:
    -----------
    trade_data : pd.DataFrame
        Raw trade data
    sample_size : int, optional
        If provided, will use a random sample of this size for faster processing
    """
    # Sample data if requested (useful for testing)
    if sample_size and len(trade_data) > sample_size:
        print(f"Using random sample of {sample_size:,} trades out of {len(trade_data):,}")
        trade_data = trade_data.sample(sample_size, random_state=42)
    
    print("\n" + "="*80)
    print("DATA PREPROCESSING")
    print("="*80)
    
    if trade_data is None or len(trade_data) == 0:
        print("No trade data to preprocess")
        return None
    
    # Create a copy to avoid modifying the original
    df = trade_data.copy()
    
    # 1. Handle missing values
    initial_rows = len(df)
    print(f"Initial rows: {initial_rows:,}")
    
    # Filter out protocol accounts if requested
    if exclude_protocols and (('trader_id' in df.columns) or ('maker_id' in df.columns) or ('taker_id' in df.columns)):
        before_filter = len(df)
        
        # Filter based on which columns are available
        if 'trader_id' in df.columns:
            df = df[~df['trader_id'].isin(PROTOCOL_ACCOUNTS)]
        
        if 'maker_id' in df.columns:
            df = df[~df['maker_id'].isin(PROTOCOL_ACCOUNTS)]
        
        if 'taker_id' in df.columns:
            df = df[~df['taker_id'].isin(PROTOCOL_ACCOUNTS)]
        
        filtered_rows = before_filter - len(df)
        print(f"Removed {filtered_rows:,} rows from protocol/exchange accounts")
        
    # Check for missing values in key columns
    missing_values = df.isnull().sum()
    missing_cols = missing_values[missing_values > 0]
    if len(missing_cols) > 0:
        print("\nMissing values in key columns:")
        for col, missing in missing_cols.items():
            print(f"  {col}: {missing:,} ({missing/len(df)*100:.1f}%)")
    
    # Drop rows with missing critical values
    critical_columns = ['trader_id']
    if any(col in df.columns for col in critical_columns):
        present_critical = [col for col in critical_columns if col in df.columns]
        df = df.dropna(subset=present_critical)
        print(f"Rows after dropping missing critical values: {len(df):,}")
    
    # 2. Handle timestamps
    if 'timestamp' in df.columns:
        if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
            print("Converting timestamps to datetime...")
            try:
                df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
                df = df.dropna(subset=['timestamp'])
                print(f"Converted {len(df):,} timestamps")
            except Exception as e:
                print(f"Error converting timestamps: {e}")
                # Create a sequential index if conversion fails
                print("Creating sequential timestamps instead")
                df = df.sort_index()
                df['timestamp'] = pd.Series(range(len(df)))
        
        # Sort by timestamp
        df = df.sort_values('timestamp')
    
    # 3. Normalize trader IDs
    if 'maker_id' in df.columns and 'taker_id' in df.columns:
        # Create trader_id from both maker and taker
        initial_rows = len(df)
        print(f"Initial rows: {initial_rows:,}")

        # Create separate records for maker and taker trades
        print("Creating separate records for maker and taker participants...")
        maker_trades = df.copy()
        maker_trades['trader_id'] = maker_trades['maker_id']
        maker_trades['trader_role'] = 'maker'

        taker_trades = df.copy()
        taker_trades['trader_id'] = taker_trades['taker_id']
        taker_trades['trader_role'] = 'taker'

        df = pd.concat([maker_trades, taker_trades], ignore_index=True)
        duplicated_rows = len(df) - initial_rows
        print(f"Split {initial_rows:,} trades into {len(df):,} trader records (added {duplicated_rows:,} rows)")
        print("Created trader_id from both maker_id and taker_id")
        
    elif 'maker_id' in df.columns and 'trader_id' not in df.columns:
        df['trader_id'] = df['maker_id']
        print("Created trader_id from maker_id")
    elif 'maker' in df.columns and 'trader_id' not in df.columns:
        df['trader_id'] = df['maker']
        print("Created trader_id from maker column")

    # Check if there's a trader_id column now
    if 'trader_id' not in df.columns:
        print("Warning: No trader_id column available")
    else:
        # Convert trader_id to string type for consistency
        df['trader_id'] = df['trader_id'].astype(str)
        unique_traders = df['trader_id'].nunique()
        print(f"Unique traders identified: {unique_traders:,}")
    
    # 4. Normalize trade amounts
    # Check if we need to scale trade amounts
    if 'trade_amount' in df.columns:
        # Check if values are extremely large (likely in base units)
        median_value = df['trade_amount'].median()
        
        if median_value > 10000:  # Threshold suggesting base units
            scaling_factor = 1e6  # Standard scaling for USDC/USD
            print(f"Scaling trade_amount by factor of {scaling_factor:,.0f}")
            
            # Store original values
            df['trade_amount_original'] = df['trade_amount']
            
            # Scale values
            df['trade_amount'] = df['trade_amount'] / scaling_factor
    elif 'size' in df.columns and 'trade_amount' not in df.columns:
        # Convert size to numeric if needed
        df['size'] = pd.to_numeric(df['size'], errors='coerce')
        
        # Check if values are extremely large
        median_value = df['size'].median()
        
        if median_value > 10000:  # Threshold suggesting base units
            scaling_factor = 1e6  # Standard scaling for USDC/USD
            print(f"Creating trade_amount from size with scaling factor of {scaling_factor:,.0f}")
            
            # Create scaled trade_amount
            df['trade_amount'] = df['size'] / scaling_factor
        else:
            # Use size directly
            print("Creating trade_amount from size (no scaling needed)")
            df['trade_amount'] = df['size']
    else:
        print("Warning: No trade_amount or size column available")
        # Create a default trade_amount column if needed
        df['trade_amount'] = 1.0
        print("Created default trade_amount column with value 1.0")
    
    # 5. Add price change column if price exists
    if 'price' in df.columns:
        # Convert price to numeric
        df['price'] = pd.to_numeric(df['price'], errors='coerce')
        
        # Calculate price changes
        df['price_change'] = df['price'].diff()
        
        # Calculate summary statistics
        print("\nPrice statistics:")
        print(f"  Min: {df['price'].min():.6f}")
        print(f"  Max: {df['price'].max():.6f}")
        print(f"  Mean: {df['price'].mean():.6f}")
        print(f"  Std Dev: {df['price'].std():.6f}")
    
    # Print summary of preprocessing
    print("\nPreprocessing complete:")
    print(f"Initial rows: {initial_rows:,}")
    print(f"Final rows: {len(df):,}")
    print(f"Dropped rows: {initial_rows - len(df):,} ({(initial_rows - len(df))/initial_rows*100:.1f}%)")
    
    return df

In [ ]:
def calculate_gini(values):
    """
    Calculate Gini coefficient for an array of values
    
    Parameters:
    -----------
    values : array-like
        Array of values (e.g., trader volumes)
        
    Returns:
    --------
    float
        Gini coefficient (0 = perfect equality, 1 = perfect inequality)
    """
    # Handle edge cases
    if len(values) <= 1 or np.sum(values) == 0:
        return 0
    
    # Sort values
    sorted_values = np.sort(values)
    n = len(sorted_values)
    
    # Calculate cumulative sum
    cumsum = np.cumsum(sorted_values)
    
    # Calculate Gini coefficient using the formula
    return (n + 1 - 2 * np.sum((n + 1 - np.arange(1, n+1)) * sorted_values) / np.sum(sorted_values)) / n

# 4. Analysis Functions

## a. Trader Distribution

In [ ]:
def analyze_trader_distribution(trade_data, config, save_prefix='trader_distribution'):
    """
    Analyze trader distribution patterns
    
    Parameters:
    -----------
    trade_data : pd.DataFrame
        DataFrame with trade data
    config : dict
        Analysis configuration
    save_prefix : str
        Prefix for saved files
        
    Returns:
    --------
    dict
        Dictionary with analysis results
    """
    print("\n" + "="*80)
    print("TRADER DISTRIBUTION ANALYSIS")
    print("="*80)
    from scipy import stats

    if trade_data is None or len(trade_data) == 0:
        print("No trade data available for analysis")
        return None
    
    results_dir = config['results_dir']
    os.makedirs(results_dir, exist_ok=True)
    
    # Calculate trader-level metrics
    trader_metrics = trade_data.groupby('trader_id').agg({
        'trade_amount': ['sum', 'mean', 'count'],
        'price': ['mean', 'std'] if 'price' in trade_data.columns else None
    })
    
    # Flatten column names
    trader_metrics.columns = [
        f"{col[0]}_{col[1]}" if col[1] else col[0] 
        for col in trader_metrics.columns
    ]
    
    # Reset index to make trader_id a column
    trader_metrics = trader_metrics.reset_index()
    
    # Calculate key statistics
    total_traders = len(trader_metrics)
    total_volume = trader_metrics['trade_amount_sum'].sum()
    avg_trades_per_trader = trader_metrics['trade_amount_count'].mean()
    median_trades_per_trader = trader_metrics['trade_amount_count'].median()
    
    print(f"Total traders: {total_traders:,}")
    print(f"Total volume: {total_volume:,.2f}")
    print(f"Average trades per trader: {avg_trades_per_trader:.2f}")
    print(f"Median trades per trader: {median_trades_per_trader:.0f}")
    
    # Create visualizations if enabled
    if config['generate_plots']:
        # 1. Trade count distribution
        plt.figure(figsize=(12, 8))
        
        # Use log scale for better visualization
        log_counts = np.log10(trader_metrics['trade_amount_count'] + 1)  # +1 to handle zeros
        
        plt.hist(log_counts, bins=50, alpha=0.7, color='skyblue')
        plt.title('Trader Activity Distribution (Log Scale)')
        plt.xlabel('Log10(Number of Trades)')
        plt.ylabel('Number of Traders')
        plt.grid(alpha=0.3)
        
        # Save and display plot
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, f"{save_prefix}_activity.png"), dpi=300)
        plt.show()
        
        # 2. Volume distribution
        plt.figure(figsize=(12, 8))
        
        # Use log scale for better visualization
        log_volumes = np.log10(trader_metrics['trade_amount_sum'] + 1)  # +1 to handle zeros
        
        plt.hist(log_volumes, bins=50, alpha=0.7, color='green')
        plt.title('Trader Volume Distribution (Log Scale)')
        plt.xlabel('Log10(Trading Volume)')
        plt.ylabel('Number of Traders')
        plt.grid(alpha=0.3)
        
        # Save and display plot
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, f"{save_prefix}_volume.png"), dpi=300)
        plt.show()
    
    # Create summary statistics for return
    deciles = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99]
    volume_percentiles = {
        f"{p}th_percentile": trader_metrics['trade_amount_sum'].quantile(p/100)
        for p in deciles
    }
    
    activity_percentiles = {
        f"{p}th_percentile": trader_metrics['trade_amount_count'].quantile(p/100)
        for p in deciles
    }
    
    summary = {
        'total_traders': total_traders,
        'total_volume': total_volume,
        'avg_trades_per_trader': avg_trades_per_trader,
        'median_trades_per_trader': median_trades_per_trader,
        'volume_percentiles': volume_percentiles,
        'activity_percentiles': activity_percentiles
    }
    if 'trade_amount' in trader_metrics.columns:
        # Log-transform for power law testing
        log_volumes = np.log10(trader_metrics['trade_amount_sum'] + 1)
        # Fit power law
        power_law_alpha, _, power_law_r, p_value, _ = stats.linregress(
            np.log10(np.arange(1, len(log_volumes) + 1)), 
            np.sort(log_volumes)[::-1]
        )
        summary['distribution_tests'] = {
            'power_law_alpha': float(power_law_alpha),
            'power_law_r_squared': float(power_law_r ** 2),
            'power_law_p_value': float(p_value),
            'follows_power_law': float(p_value) < 0.05 and power_law_r ** 2 > 0.8
        }
    
    # Save summary if enabled
    if config['save_results']:
        with open(os.path.join(results_dir, f"{save_prefix}_summary.json"), 'w') as f:
            json.dump(summary, f, indent=2, default=str)
    
    return summary, trader_metrics


## b. Whale Identification


In [ ]:
# Fix for the identify_whales function's Lorenz curve plotting
def identify_whales(trade_data, config, save_prefix='whale_identification'):
    """
    Identify whale traders based on specified criteria
    
    Parameters:
    -----------
    trade_data : pd.DataFrame
        DataFrame with trade data
    config : dict
        Analysis configuration
    save_prefix : str
        Prefix for saved files
        
    Returns:
    --------
    tuple
        (whale_ids, whale_results) - List of whale IDs and analysis results
    """
    print("\n" + "="*80)
    print("WHALE TRADER IDENTIFICATION")
    print("="*80)
    
    if trade_data is None or len(trade_data) == 0:
        print("No trade data available for analysis")
        return [], {}
    
    # Ensure we have the necessary columns
    if 'trader_id' not in trade_data.columns or 'trade_amount' not in trade_data.columns:
        print("Error: Missing required columns (trader_id, trade_amount)")
        return [], {}
    
    # Get configuration parameters
    threshold = config['whale_threshold']
    results_dir = config['results_dir']
    generate_plots = config['generate_plots']
    
    os.makedirs(results_dir, exist_ok=True)
    
    # Group trades by trader and calculate total volume
    trader_volumes = trade_data.groupby('trader_id')['trade_amount'].sum().sort_values(ascending=False)
    
    # Calculate total volume
    total_volume = trader_volumes.sum()
    total_traders = len(trader_volumes)
    
    print(f"Total traders: {total_traders:,}")
    print(f"Total volume: {total_volume:,.2f}")
    
    # Create cumulative volume percentages
    cumulative_volumes = trader_volumes.cumsum()
    cumulative_percentages = cumulative_volumes / total_volume * 100
    
    # Create DataFrame for analysis
    trader_analysis = pd.DataFrame({
        'trader_id': trader_volumes.index,
        'volume': trader_volumes.values,
        'cumulative_volume': cumulative_volumes.values,
        'volume_pct': trader_volumes.values / total_volume * 100,
        'cumulative_pct': cumulative_percentages.values
    })
    
    # Calculate Gini coefficient
    gini = calculate_gini(trader_volumes.values)
    print(f"Volume concentration (Gini coefficient): {gini:.4f}")
    
    # Define percentile thresholds to evaluate
    percentile_thresholds = [0.001, 0.01, 0.05, 0.1]
    
    # Calculate metrics for each threshold
    threshold_metrics = []
    for pct in percentile_thresholds:
        num_whales = max(1, int(total_traders * pct))
        whale_volume = trader_volumes.iloc[:num_whales].sum()
        whale_volume_pct = whale_volume / total_volume * 100
        
        # Store metrics
        threshold_metrics.append({
            'threshold': pct,
            'threshold_label': f"Top {pct*100:.1f}%",
            'num_whales': num_whales,
            'whale_volume': float(whale_volume),
            'whale_volume_pct': float(whale_volume_pct),
            'trader_pct': float(num_whales / total_traders * 100)
        })
        
        print(f"Top {pct*100:.1f}% definition ({num_whales:,} traders): {whale_volume_pct:.2f}% of volume")
    
    # Calculate volume coverage thresholds
    volume_thresholds = [50, 75, 90, 95]
    coverage_metrics = []
    
    for pct in volume_thresholds:
        # Find traders needed to reach this volume percentage
        traders_needed = sum(cumulative_percentages < pct) + 1
        traders_needed = min(traders_needed, len(trader_volumes))
        
        # Get the actual volume percentage
        actual_pct = cumulative_percentages.iloc[traders_needed-1] if traders_needed <= len(cumulative_percentages) else 100
        
        coverage_metrics.append({
            'volume_threshold': pct,
            'threshold_label': f"{pct}% Volume",
            'num_traders': int(traders_needed),
            'actual_volume_pct': float(actual_pct),
            'trader_pct': float(traders_needed / total_traders * 100)
        })
        
        print(f"Traders needed for {pct}% volume: {traders_needed:,} ({traders_needed/total_traders*100:.4f}% of all traders)")
    
    # Create visualizations
    # Fix for the Lorenz curve plotting
    if generate_plots:
        # Create figure for combined plots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # 1. Lorenz curve - Fixed to ensure x and y arrays have same dimension
        # Calculate cumulative percentages of traders and volumes
        trader_pcts = np.linspace(0, 100, len(trader_volumes) + 1)
        volume_pcts = np.concatenate(([0], np.cumsum(trader_volumes.values) / total_volume * 100))
        
        # Fix for Lorenz curve visualization
        ax1.plot(trader_pcts, volume_pcts, 'b-', linewidth=2, label='Volume distribution')
        ax1.plot([0, 100], [0, 100], 'k--', label='Perfect equality')
        ax1.fill_between(trader_pcts, volume_pcts, trader_pcts, alpha=0.2)

        # Add key percentiles with better positioning
        percentile_positions = {
            99.9: {'x_shift': 1, 'y_shift': 5},
            99: {'x_shift': 1, 'y_shift': 15},
            95: {'x_shift': 1.5, 'y_shift': 25},
            90: {'x_shift': 2, 'y_shift': 35}
        }

        for p, shifts in percentile_positions.items():
            # Calculate index for this percentile
            idx = min(int(total_traders * (100-p)/100), len(trader_volumes)-1)
            if idx >= 0:
                # Get x and y coordinates
                x = (idx / total_traders) * 100
                y = volume_pcts[idx+1] if idx+1 < len(volume_pcts) else 100
                
                # Add reference lines
                ax1.plot([x, x], [0, y], 'r--', alpha=0.5)
                ax1.plot([0, x], [y, y], 'r--', alpha=0.5)
                
                # Add label with custom positioning
                ax1.text(x + shifts['x_shift'], y - shifts['y_shift'], 
                        f'Top {100-p:.1f}%: {y:.1f}%', 
                        fontsize=9,
                        bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))
        
        ax1.set_title(f'Trading Volume Distribution (Gini: {gini:.4f})')
        ax1.set_xlabel('Cumulative % of Traders')
        ax1.set_ylabel('Cumulative % of Volume')
        ax1.grid(alpha=0.3)
        ax1.legend()
        
        # 2. Whale definition comparison
        percent_definitions = pd.DataFrame(threshold_metrics)
        
        # Plot bars for percentage of traders vs percentage of volume
        bar_width = 0.35
        x = np.arange(len(percent_definitions))
        
        ax2.bar(x - bar_width/2, percent_definitions['trader_pct'], 
               bar_width, label='% of Traders', color='skyblue')
        ax2.bar(x + bar_width/2, percent_definitions['whale_volume_pct'], 
               bar_width, label='% of Volume', color='orange')
        
        # Set x-axis labels
        ax2.set_xticks(x)
        ax2.set_xticklabels(percent_definitions['threshold_label'])
        
        # Add value labels on bars
        for i, v in enumerate(percent_definitions['trader_pct']):
            ax2.text(i - bar_width/2, v + 1, f"{v:.2f}%", ha='center', fontsize=9)
        
        for i, v in enumerate(percent_definitions['whale_volume_pct']):
            ax2.text(i + bar_width/2, v + 1, f"{v:.2f}%", ha='center', fontsize=9)
        
        ax2.set_title('Whale Definitions Comparison')
        ax2.set_ylabel('Percentage')
        ax2.set_ylim(0, 100)
        ax2.grid(axis='y', alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, f"{save_prefix}_analysis.png"), dpi=300)
        plt.show()
        
        print(f"Whale definition analysis visualizations saved to {save_prefix}_analysis.png")
    
    # Use the specified threshold
    num_whales = max(1, int(total_traders * threshold))
    whale_ids = trader_volumes.head(num_whales).index.tolist()
    
    print(f"\nUsing top {threshold*100:.1f}% definition: {num_whales:,} whales")
    print(f"Selected whale threshold volume: {trader_volumes.iloc[num_whales-1] if num_whales <= len(trader_volumes) else 0:.2f}")
    
    # Save results if enabled
    results = {
        'gini_coefficient': gini,
        'threshold_used': threshold,
        'num_whales': num_whales,
        'threshold_metrics': threshold_metrics,
        'coverage_metrics': coverage_metrics,
        'whale_volume_percentage': float(trader_volumes.head(num_whales).sum() / total_volume * 100)
    }
    
    if config['save_results']:
        with open(os.path.join(results_dir, f"{save_prefix}_results.json"), 'w') as f:
            json.dump(results, f, indent=2, default=str)
    
    # Return whale IDs and analysis results
    return whale_ids, results

## d. Trader Classification

In [ ]:
def run_trader_classification_analysis(trade_data, min_clusters=2, max_clusters=5, random_state=42, save_dir='results/trader_analysis', max_traders=None):
    """
    Analyze how different trader types behave based on trading patterns - optimized for speed
    """
    # Validate input
    if trade_data is None or len(trade_data) == 0:
        print("Error: Empty trade data")
        return None
    
    # Ensure trader_id exists
    if 'trader_id' not in trade_data.columns:
        print("Error: No trader_id column found")
        return None
    
    # Create output directory
    os.makedirs(save_dir, exist_ok=True)
    
    start_time = time.time()
    print(f"Total trades: {len(trade_data):,}")
    
    # Filter out protocol accounts if defined
    if 'PROTOCOL_ACCOUNTS' in globals():
        trade_data = trade_data[~trade_data['trader_id'].isin(PROTOCOL_ACCOUNTS)]
        print(f"Filtered out protocol accounts, remaining trades: {len(trade_data):,}")
    
    # OPTIMIZATION: Calculate all aggregations at once with a single groupby operation
    print("Calculating trader metrics...")
    agg_funcs = {
        'trade_amount': ['sum', 'mean'] if 'trade_amount' in trade_data.columns else [],
        'price': ['mean', 'std'] if 'price' in trade_data.columns else []
    }
    
    # Add count operation (works on any column)
    count_col = 'trader_id'
    if count_col not in agg_funcs:
        agg_funcs[count_col] = ['count']
    
    # Handle buy ratio calculation
    if 'side' in trade_data.columns:
        # Add is_buy column
        trade_data['is_buy'] = trade_data['side'].str.lower() == 'buy'
        agg_funcs['is_buy'] = ['mean']
    
    # OPTIMIZATION: Use one groupby operation instead of multiple
    trader_aggs = trade_data.groupby('trader_id').agg(agg_funcs)
    
    # Flatten column names
    trader_aggs.columns = ['_'.join(col).strip('_') for col in trader_aggs.columns.values]
    
    # Rename columns for clarity
    column_mapping = {
        f'{count_col}_count': 'trade_count',
        'trade_amount_sum': 'total_volume',
        'trade_amount_mean': 'avg_trade_size',
        'price_mean': 'avg_price',
        'price_std': 'price_std',
        'is_buy_mean': 'buy_ratio'
    }
    
    # Apply renaming only for columns that exist
    rename_dict = {old: new for old, new in column_mapping.items() if old in trader_aggs.columns}
    trader_aggs = trader_aggs.rename(columns=rename_dict)
    
    # Calculate price volatility (std/mean)
    if 'price_std' in trader_aggs.columns and 'avg_price' in trader_aggs.columns:
        trader_aggs['price_volatility'] = trader_aggs['price_std'] / trader_aggs['avg_price']
        # Clean up infinite values
        trader_aggs['price_volatility'] = trader_aggs['price_volatility'].replace([np.inf, -np.inf], np.nan)
        trader_aggs['price_volatility'] = trader_aggs['price_volatility'].fillna(0)
    
    # OPTIMIZATION: Calculate trade frequency for all traders at once if timestamp exists
    if 'timestamp' in trade_data.columns and pd.api.types.is_datetime64_any_dtype(trade_data['timestamp']):
        print("Calculating trading frequencies...")
        # Get min and max timestamp per trader
        time_range = trade_data.groupby('trader_id')['timestamp'].agg(['min', 'max'])
        # Calculate duration in days
        time_range['duration_days'] = (time_range['max'] - time_range['min']).dt.total_seconds() / 86400
        # Fix any zero durations to avoid division by zero
        time_range['duration_days'] = time_range['duration_days'].replace(0, 1)
        # Join trade count and calculate frequency
        time_range = time_range.join(trader_aggs[['trade_count']])
        time_range['trade_frequency'] = time_range['trade_count'] / time_range['duration_days']
        # Add to main aggregations
        trader_aggs['trade_frequency'] = time_range['trade_frequency']
    else:
        trader_aggs['trade_frequency'] = 0
    
    # Convert to DataFrame with trader_id as a column
    trader_df = trader_aggs.reset_index()
    
    # OPTIMIZATION: Limit to traders with at least 3 trades for clustering
    trader_df = trader_df[trader_df['trade_count'] >= 3]
    
    # OPTIMIZATION: Sample traders if there are too many
    if max_traders and len(trader_df) > max_traders:
        print(f"Sampling {max_traders:,} traders out of {len(trader_df):,} for faster analysis")
        trader_df = trader_df.sample(max_traders, random_state=random_state)
    
    print(f"\nTrader features DataFrame: {len(trader_df):,} traders")
    
    # Select features for clustering
    cluster_features = [col for col in ['trade_count', 'total_volume', 'avg_trade_size', 
                                       'trade_frequency', 'price_volatility', 'buy_ratio'] 
                       if col in trader_df.columns]
    
    print(f"Cluster features: {cluster_features}")
    
    if len(cluster_features) < 2:
        print(f"Error: Not enough features for clustering, found only: {cluster_features}")
        return None
    
    # OPTIMIZATION: Directly create numpy array for clustering
    X = trader_df[cluster_features].values
    
    # OPTIMIZATION: Use faster log transform
    X_log = np.log1p(X)
    
    # OPTIMIZATION: Scale the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_log)
    
    # Determine optimal number of clusters
    optimal_clusters = min(max_clusters, max(min_clusters, len(trader_df) // 10000 + 2))
    
    print(f"Using {optimal_clusters} clusters")
    
    # OPTIMIZATION: Use MiniBatchKMeans for large datasets
    if len(trader_df) > 10000:
        from sklearn.cluster import MiniBatchKMeans
        kmeans = MiniBatchKMeans(n_clusters=optimal_clusters, random_state=random_state, 
                                 batch_size=1000, n_init='auto')
    else:
        kmeans = KMeans(n_clusters=optimal_clusters, random_state=random_state, n_init=10)
    
    trader_df['cluster'] = kmeans.fit_predict(X_scaled)
    
    # Calculate cluster summaries
    print("Analyzing clusters...")
    cluster_summary = trader_df.groupby('cluster').agg({
        'trade_count': ['mean', 'median', 'min', 'max', 'count'],
        'total_volume': ['sum', 'mean', 'median'] if 'total_volume' in trader_df.columns else 'count',
        'avg_trade_size': ['mean', 'median', 'min', 'max'] if 'avg_trade_size' in trader_df.columns else 'count',
    })
    
    # Flatten MultiIndex columns
    cluster_summary.columns = ['_'.join(col).strip('_') for col in cluster_summary.columns.values]
    
    # Save summary to CSV
    cluster_summary.to_csv(os.path.join(save_dir, 'cluster_summary.csv'))
    
    # OPTIMIZATION: Simplified cluster characteristics
    cluster_stats = {}
    cluster_types = {}
    small_clusters = []
    
    for cluster_id in range(optimal_clusters):
        cluster_data = trader_df[trader_df['cluster'] == cluster_id]
        
        if len(cluster_data) == 0:
            continue
        
        # Calculate essential metrics
        stats = {
            'trader_count': len(cluster_data),
            'avg_trade_count': float(cluster_data['trade_count'].mean()),
            'total_volume': float(cluster_data['total_volume'].sum()) if 'total_volume' in cluster_data.columns else 0,
            'avg_trade_size': float(cluster_data['avg_trade_size'].mean()) if 'avg_trade_size' in cluster_data.columns else 0,
            'avg_trade_frequency': float(cluster_data['trade_frequency'].mean()) if 'trade_frequency' in cluster_data.columns else 0,
            'avg_price_volatility': float(cluster_data['price_volatility'].mean()) if 'price_volatility' in cluster_data.columns else 0,
            'avg_buy_ratio': float(cluster_data['buy_ratio'].mean()) if 'buy_ratio' in cluster_data.columns else 0.5,
            'example_traders': cluster_data['trader_id'].head(5).tolist()
        }
        
        cluster_stats[cluster_id] = stats
        
        # Determine trader type based on characteristics
        trader_type = "Retail Traders"  # Default
        
        total_volume = stats['total_volume']
        avg_trade_count = stats['avg_trade_count']
        avg_trade_frequency = stats['avg_trade_frequency']
        
        if total_volume > trader_df['total_volume'].quantile(0.95):
            trader_type = "Whale Traders"
        elif avg_trade_count > trader_df['trade_count'].quantile(0.9):
            trader_type = "High-Frequency Traders"
        elif avg_trade_frequency > trader_df['trade_frequency'].quantile(0.9):
            trader_type = "Momentum Traders"
        
        cluster_types[cluster_id] = trader_type
        
        # Track small clusters
        if stats['trader_count'] < 10:
            small_clusters.append({
                'cluster_id': cluster_id,
                'trader_count': stats['trader_count'],
                'trader_type': trader_type,
                'trader_ids': cluster_data['trader_id'].tolist()
            })
    
    # Print brief summary
    print("\nCluster Summary:")
    for cluster_id, stats in cluster_stats.items():
        print(f"Cluster {cluster_id} ({cluster_types[cluster_id]}): {stats['trader_count']} traders")
    
    # Save enhanced cluster details
    cluster_detail = pd.DataFrame([
        {
            'cluster': cluster_id,
            'trader_count': stats['trader_count'],
            'trader_percentage': 100 * stats['trader_count'] / len(trader_df),
            'trader_type': cluster_types[cluster_id],
            'avg_trade_count': stats['avg_trade_count'],
            'total_volume': stats['total_volume'],
            'avg_trade_size': stats['avg_trade_size'],
            'avg_trade_frequency': stats['avg_trade_frequency'],
            'avg_price_volatility': stats['avg_price_volatility'],
            'avg_buy_ratio': stats['avg_buy_ratio'],
            'example_trader_ids': ', '.join(stats['example_traders'])
        }
        for cluster_id, stats in cluster_stats.items()
    ])
    
    cluster_detail.to_csv(os.path.join(save_dir, 'cluster_details.csv'), index=False)
    
    # Create full trader features with types
    trader_df['trader_type'] = trader_df['cluster'].map(cluster_types)
    
    # Save essential data
    trader_df.to_csv(os.path.join(save_dir, 'trader_features.csv'), index=False)
    
    # Output small cluster trader IDs
    if small_clusters:
        small_cluster_details = []
        print("\nSmall clusters detected (< 10 traders):")
        
        for cluster in small_clusters:
            print(f"Cluster {cluster['cluster_id']} ({cluster['trader_type']}): {cluster['trader_count']} traders")
            print(f"Trader IDs: {', '.join(cluster['trader_ids'])}")
            
            for trader_id in cluster['trader_ids']:
                small_cluster_details.append({
                    'cluster_id': cluster['cluster_id'],
                    'trader_type': cluster['trader_type'],
                    'trader_id': trader_id
                })
        
        pd.DataFrame(small_cluster_details).to_csv(os.path.join(save_dir, 'small_cluster_traders.csv'), index=False)
    
    # OPTIMIZATION: Only create visualizations if there's a reasonable number of traders
    if len(trader_df) <= 50000:
        print("Creating visualizations...")
        # Create PCA visualization
        features_for_viz = trader_df[cluster_features].copy()
        features_for_viz = (features_for_viz - features_for_viz.mean()) / features_for_viz.std()
        
        pca = PCA(n_components=2)
        pca_result = pca.fit_transform(features_for_viz.values)
        
        pca_df = pd.DataFrame({
            'trader_id': trader_df['trader_id'],
            'trader_type': trader_df['trader_type'],
            'PCA1': pca_result[:, 0],
            'PCA2': pca_result[:, 1]
        })
        
        plt.figure(figsize=(12, 8))
        
        for trader_type, group in pca_df.groupby('trader_type'):
            # OPTIMIZATION: Sample points for large groups
            if len(group) > 1000:
                group = group.sample(1000, random_state=random_state)
                
            plt.scatter(
                group['PCA1'], 
                group['PCA2'], 
                alpha=0.6, 
                label=f"{trader_type} (n={len(group):,})",
                s=50
            )
        
        plt.title('Trader Type Clusters', fontsize=16)
        plt.xlabel('PCA Component 1', fontsize=12)
        plt.ylabel('PCA Component 2', fontsize=12)
        plt.grid(alpha=0.3)
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, 'trader_clusters_pca.png'), dpi=300)
        plt.close()
        
        # Create pie chart visualization
        type_counts = trader_df['trader_type'].value_counts()
        type_pcts = 100 * type_counts / type_counts.sum()
        
        significant_types = type_pcts[type_pcts > 0.5]
        
        plt.figure(figsize=(10, 8))
        plt.pie(
            significant_types.values, 
            labels=[f"{t}\n{p:.1f}%" for t, p in zip(significant_types.index, significant_types.values)],
            autopct=lambda p: f"{p:.1f}%" if p > 3 else '',
            startangle=90,
            explode=[0.05] * len(significant_types),
            shadow=True
        )
        plt.title('Trader Type Distribution', fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, 'trader_type_distribution.png'), dpi=300)
        plt.close()
    else:
        print("Skipping visualizations due to large dataset size")
    
    elapsed = time.time() - start_time
    print(f"Classified {len(trader_df):,} traders into {optimal_clusters} clusters")
    print(f"Completed Trader Classification in {elapsed:.1f} seconds")
    
    return {
        'trader_features': trader_df,
        'cluster_summary': cluster_summary,
        'cluster_details': cluster_detail,
        'trader_types': cluster_types,
        'small_clusters': small_clusters if small_clusters else None
    }

## e. Market Dynamics

In [ ]:

# 5.4 Market Dynamics Analysis
def analyze_market_dynamics(trade_data, whale_ids, market_data, config, save_prefix='market_dynamics'):
    """
    Analyze market dynamics including whale impact and price movements
    
    Parameters:
    -----------
    trade_data : pd.DataFrame
        DataFrame with trade data
    whale_ids : list
        List of whale trader IDs
    market_data : pd.DataFrame
        DataFrame with market metadata
    config : dict
        Analysis configuration
    save_prefix : str
        Prefix for saved files
        
    Returns:
    --------
    dict
        Dictionary with market dynamics analysis results
    """
    print("\n" + "="*80)
    print("MARKET DYNAMICS ANALYSIS")
    print("="*80)
    
    if trade_data is None or len(trade_data) == 0:
        print("No trade data available for analysis")
        return None
    
    if not whale_ids:
        print("No whale traders identified for analysis")
        return None
    
    results_dir = config['results_dir']
    generate_plots = config['generate_plots']
    
    os.makedirs(results_dir, exist_ok=True)
    
    # Clean the data
    df = trade_data.copy()
    
    # Add whale indicator
    df['is_whale'] = df['trader_id'].isin(whale_ids)
    
    # Ensure price is numeric
    if 'price' in df.columns:
        df['price'] = pd.to_numeric(df['price'], errors='coerce')
    
    # Split whale and non-whale trades
    whale_trades = df[df['is_whale']]
    non_whale_trades = df[~df['is_whale']]
    
    print(f"Analyzing {len(whale_trades)} whale trades and {len(non_whale_trades)} non-whale trades")
    
    # Market-level price impact analysis
    if 'market_id' in df.columns and 'price' in df.columns:
        market_impacts = []
        
        for market_id, market_df in df.groupby('market_id'):
            # Skip markets with too few trades
            if len(market_df) < 10:
                continue
                
            # Sort by timestamp
            if 'timestamp' in market_df.columns:
                if not pd.api.types.is_datetime64_any_dtype(market_df['timestamp']):
                    try:
                        market_df['timestamp'] = pd.to_datetime(market_df['timestamp'], errors='coerce')
                        market_df = market_df.dropna(subset=['timestamp'])
                    except:
                        # Create a sequential timestamp if conversion fails
                        market_df = market_df.sort_index()
                        market_df['timestamp'] = pd.Series(range(len(market_df)))
                
                market_df = market_df.sort_values('timestamp')
            
            # Calculate price changes
            market_df['price_change'] = market_df['price'].diff()
            
            # Skip markets with no price changes
            if market_df['price_change'].isna().all() or market_df['price_change'].abs().sum() == 0:
                continue
            
            # Separate whale and non-whale trades
            market_whale_trades = market_df[market_df['is_whale']]
            market_non_whale_trades = market_df[~market_df['is_whale']]
            
            # Skip markets with no whale trades
            if len(market_whale_trades) == 0:
                continue
                
            # Calculate average metrics
            whale_avg_change = market_whale_trades['price_change'].mean()
            non_whale_avg_change = market_non_whale_trades['price_change'].mean()
            
            # Calculate directional impact with magnitude
            if len(market_whale_trades) > 0:
                positive_whale_changes = market_whale_trades[market_whale_trades['price_change'] > 0]['price_change']
                negative_whale_changes = market_whale_trades[market_whale_trades['price_change'] < 0]['price_change']
                
                # Calculate percentage by count
                whale_pos_pct = (market_whale_trades['price_change'] > 0).mean() * 100
                whale_neg_pct = (market_whale_trades['price_change'] < 0).mean() * 100
                
                # Add magnitude metrics
                whale_pos_avg_magnitude = positive_whale_changes.mean() if len(positive_whale_changes) > 0 else 0
                whale_neg_avg_magnitude = abs(negative_whale_changes.mean()) if len(negative_whale_changes) > 0 else 0
            else:
                whale_pos_pct = 0
                whale_neg_pct = 0
                whale_pos_avg_magnitude = 0
                whale_neg_avg_magnitude = 0

            # Same for non-whale trades
            if len(market_non_whale_trades) > 0:
                positive_non_whale_changes = market_non_whale_trades[market_non_whale_trades['price_change'] > 0]['price_change']
                negative_non_whale_changes = market_non_whale_trades[market_non_whale_trades['price_change'] < 0]['price_change']
                
                non_whale_pos_pct = (market_non_whale_trades['price_change'] > 0).mean() * 100
                non_whale_neg_pct = (market_non_whale_trades['price_change'] < 0).mean() * 100
                
                non_whale_pos_avg_magnitude = positive_non_whale_changes.mean() if len(positive_non_whale_changes) > 0 else 0
                non_whale_neg_avg_magnitude = abs(negative_non_whale_changes.mean()) if len(negative_non_whale_changes) > 0 else 0
            else:
                non_whale_pos_pct = 0
                non_whale_neg_pct = 0
                non_whale_pos_avg_magnitude = 0
                non_whale_neg_avg_magnitude = 0

            # Add these new metrics to the market_impacts dictionary
            market_impacts.append({
                'market_id': market_id,
                'market_name': str(market_id),
                'total_trades': len(market_df),
                'whale_trades': len(market_whale_trades),
                'non_whale_trades': len(market_non_whale_trades),
                'whale_trade_pct': len(market_whale_trades) / len(market_df) * 100,
                'whale_avg_change': float(whale_avg_change),
                'non_whale_avg_change': float(non_whale_avg_change),
                'whale_pos_pct': float(whale_pos_pct),
                'whale_neg_pct': float(whale_neg_pct),
                'non_whale_pos_pct': float(non_whale_pos_pct),
                'non_whale_neg_pct': float(non_whale_neg_pct),
                'whale_pos_avg_magnitude': float(whale_pos_avg_magnitude),
                'whale_neg_avg_magnitude': float(whale_neg_avg_magnitude), 
                'non_whale_pos_avg_magnitude': float(non_whale_pos_avg_magnitude),
                'non_whale_neg_avg_magnitude': float(non_whale_neg_avg_magnitude)
            })
        
        # Create markets DataFrame
        if market_impacts:
            markets_df = pd.DataFrame(market_impacts)
            
            # Calculate weighted averages
            weighted_whale_impact = np.average(
                markets_df['whale_avg_change'],
                weights=markets_df['whale_trades']
            )
            
            weighted_non_whale_impact = np.average(
                markets_df['non_whale_avg_change'],
                weights=markets_df['non_whale_trades']
            )
            
            print(f"\nWeighted average whale price impact: {weighted_whale_impact:.6f}")
            print(f"Weighted average non-whale price impact: {weighted_non_whale_impact:.6f}")
            
            # Calculate impact ratio if possible
            if weighted_non_whale_impact != 0:
                impact_ratio = weighted_whale_impact / weighted_non_whale_impact
                print(f"Impact ratio (whale/non-whale): {impact_ratio:.4f}")
            else:
                impact_ratio = None
                print("Impact ratio cannot be calculated (division by zero)")
            
            # Create visualizations
            if generate_plots:
                plt.figure(figsize=(15, 12))
                
                # 1. Market-by-market comparison
                plt.subplot(2, 1, 1)
                
                # Sort markets by whale impact
                sorted_markets = markets_df.sort_values('whale_avg_change')
                
                # Plot whale vs non-whale impact by market
                plt.scatter(range(len(sorted_markets)), sorted_markets['whale_avg_change'], 
                           label='Whale impact', alpha=0.7, s=50, color='blue')
                plt.scatter(range(len(sorted_markets)), sorted_markets['non_whale_avg_change'], 
                           label='Non-whale impact', alpha=0.7, s=50, color='orange')
                
                plt.axhline(y=0, color='r', linestyle='--')
                plt.title('Price Impact by Market')
                plt.xlabel('Markets (sorted by whale impact)')
                plt.ylabel('Average Price Change')
                plt.legend()
                plt.grid(alpha=0.3)
                
                # 2. Direction comparison
                plt.subplot(2, 1, 2)
                
                # Calculate average positive/negative percentages
                avg_whale_pos = markets_df['whale_pos_pct'].mean()
                avg_whale_neg = markets_df['whale_neg_pct'].mean()
                avg_nonwhale_pos = markets_df['non_whale_pos_pct'].mean()
                avg_nonwhale_neg = markets_df['non_whale_neg_pct'].mean()
                
                # Plot directional impact
                labels = ['Whale', 'Non-whale']
                pos_values = [avg_whale_pos, avg_nonwhale_pos]
                neg_values = [avg_whale_neg, avg_nonwhale_neg]
                neutral_values = [100 - avg_whale_pos - avg_whale_neg, 
                                 100 - avg_nonwhale_pos - avg_nonwhale_neg]
                
                width = 0.35
                x = np.arange(len(labels))
                
                plt.bar(x, pos_values, width, label='Positive impact', color='green')
                plt.bar(x, neg_values, width, bottom=pos_values, label='Negative impact', color='red')
                plt.bar(x, neutral_values, width, 
                       bottom=[pos_values[i] + neg_values[i] for i in range(len(pos_values))], 
                       label='Neutral', color='gray')
                
                plt.title('Direction of Price Impact')
                plt.ylabel('Percentage of Trades')
                plt.xlabel('Trader Type')
                plt.xticks(x, labels)
                plt.legend()
                
                plt.tight_layout()
                plt.savefig(os.path.join(results_dir, f"{save_prefix}_price_impact.png"), dpi=300)
                plt.show()
                
                print(f"Market price impact visualization saved to {save_prefix}_price_impact.png")
            
            # Save results if enabled
            results = {
                'market_impacts': [m for m in market_impacts if isinstance(m, dict)],
                'weighted_whale_impact': float(weighted_whale_impact),
                'weighted_non_whale_impact': float(weighted_non_whale_impact),
                'impact_ratio': float(impact_ratio) if impact_ratio is not None else None,
                'avg_whale_positive_pct': float(avg_whale_pos),
                'avg_whale_negative_pct': float(avg_whale_neg),
                'avg_nonwhale_positive_pct': float(avg_nonwhale_pos),
                'avg_nonwhale_negative_pct': float(avg_nonwhale_neg)
            }
            
            if config['save_results']:
                with open(os.path.join(results_dir, f"{save_prefix}_results.json"), 'w') as f:
                    json.dump(results, f, indent=2, default=str)
            
            return results
    
    # If market-level analysis was not possible, do overall analysis
    print("Analyzing overall price changes...")
    
    # Calculate metrics
    whale_avg_change = whale_trades['price_change'].mean() if 'price_change' in whale_trades.columns else None
    whale_median_change = whale_trades['price_change'].median() if 'price_change' in whale_trades.columns else None
    whale_std_change = whale_trades['price_change'].std() if 'price_change' in whale_trades.columns else None
    
    non_whale_avg_change = non_whale_trades['price_change'].mean() if 'price_change' in non_whale_trades.columns else None
    non_whale_median_change = non_whale_trades['price_change'].median() if 'price_change' in non_whale_trades.columns else None
    non_whale_std_change = non_whale_trades['price_change'].std() if 'price_change' in non_whale_trades.columns else None
    
    if all(x is not None for x in [whale_avg_change, non_whale_avg_change]):
        print(f"\nWhale trades average price change: {whale_avg_change:.6f}")
        print(f"Non-whale trades average price change: {non_whale_avg_change:.6f}")
    else:
        print("Price change metrics not available")
        
    results = {
        'overall_metrics': {
            'whale_avg_change': float(whale_avg_change) if whale_avg_change is not None else None,
            'whale_median_change': float(whale_median_change) if whale_median_change is not None else None,
            'whale_std_change': float(whale_std_change) if whale_std_change is not None else None,
            'non_whale_avg_change': float(non_whale_avg_change) if non_whale_avg_change is not None else None,
            'non_whale_median_change': float(non_whale_median_change) if non_whale_median_change is not None else None,
            'non_whale_std_change': float(non_whale_std_change) if non_whale_std_change is not None else None
        }
    }
    
    if config['save_results']:
        with open(os.path.join(results_dir, f"{save_prefix}_results.json"), 'w') as f:
            json.dump(results, f, indent=2, default=str)
    
    return results

## f. Temporal Analaysis

In [ ]:
def analyze_trader_temporal_patterns(trade_data, trader_types_df, period_col=None, min_date=None, max_date=None):
    """
    Analyze how different trader types behave over time with improved performance
    
    Parameters:
    -----------
    trade_data : pd.DataFrame
        DataFrame with trade data including timestamps
    trader_types_df : pd.DataFrame
        DataFrame with trader classification results
    period_col : str, optional
        Column name to use for time periods (if None, will be created)
    min_date : datetime, optional
        Optional start date for analysis (if None, uses earliest date in data)
    max_date : datetime, optional
        Optional end date for analysis (if None, uses latest date in data)
        
    Returns:
    --------
    dict
        Dictionary with temporal analysis results
    """
    print("Starting temporal analysis...")
    
    # Ensure we have timestamp data
    if 'timestamp' not in trade_data.columns:
        print("Error: No timestamp data available for temporal analysis")
        return None
    
    # Ensure timestamp is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(trade_data['timestamp']):
        try:
            print("Converting timestamps to datetime format...")
            trade_data = trade_data.copy()
            trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])
            print(f"Converted {len(trade_data)} timestamps successfully")
        except Exception as e:
            print(f"Error: Could not convert timestamps to datetime: {e}")
            return None
    
    # Merge trader types to trade data (if needed)
    if 'trader_type' not in trade_data.columns:
        print("Merging trader types with trade data...")
        if ('trader_id' in trade_data.columns and 
            'trader_id' in trader_types_df.columns and 
            'trader_type' in trader_types_df.columns):
            
            # Create a more efficient merge by only bringing necessary columns
            trade_data = trade_data.merge(
                trader_types_df[['trader_id', 'trader_type']], 
                on='trader_id', 
                how='left'
            )
            
            # Check if merge worked
            if trade_data['trader_type'].isna().all():
                print("Warning: Trader type merge resulted in all NaN values")
                return None
        else:
            print("Error: Cannot merge trader types - missing required columns")
            return None
    
    # Create time periods efficiently
    # First get date range
    start_date = min_date if min_date is not None else trade_data['timestamp'].min()
    end_date = max_date if max_date is not None else trade_data['timestamp'].max()
    
    if start_date is None or end_date is None:
        print("Error: Invalid date range in trade data")
        return None
    
    # Calculate market duration in days
    market_duration = (end_date - start_date).total_seconds() / 86400
    if market_duration < 1:
        print("Warning: Market duration too short for temporal analysis")
        market_duration = 1  # Prevent division by zero
    
    # Create time period column if not provided
    if period_col is None:
        print(f"Creating time periods based on market duration: {market_duration:.1f} days")
        trade_data = trade_data.copy()
        
        # For longer markets, use weeks; for shorter ones, use days
        if market_duration >= 30:
            # Calculate week number using vectorized operations
            trade_data['week'] = ((trade_data['timestamp'] - pd.Timestamp(start_date))
                               .dt.total_seconds() / (86400 * 7)).astype(int)
            period_col = 'week'
            period_name = 'Week'
            print(f"Using weeks as period unit ({int(market_duration/7)} weeks total)")
        else:
            # Calculate day number using vectorized operations
            trade_data['day'] = ((trade_data['timestamp'] - pd.Timestamp(start_date))
                              .dt.total_seconds() / 86400).astype(int)
            period_col = 'day'
            period_name = 'Day'
            print(f"Using days as period unit ({int(market_duration)} days total)")
    else:
        # Using provided period column
        if period_col not in trade_data.columns:
            print(f"Error: Specified period column '{period_col}' not found in data")
            return None
        
        # Infer period name from column name
        period_name = period_col.capitalize()
        print(f"Using provided period column: {period_col}")
    
    # Drop rows with missing trader_type
    valid_data = trade_data.dropna(subset=['trader_type'])
    if len(valid_data) == 0:
        print("Error: No data available after filtering for temporal analysis")
        return None
    print(f"Analyzing {len(valid_data):,} trades with valid trader types")
    
    # Create a uniform dictionary to store all traders by type
    total_traders_by_type = valid_data.groupby('trader_type')['trader_id'].nunique().to_dict()
    print(f"Trader types distribution: {total_traders_by_type}")
    
    # Calculate period entry points for each trader type
    entry_periods = {}
    
    # Efficient trader entry calculation using groupby operations
    trader_entries = valid_data.groupby(['trader_id', 'trader_type'])[period_col].min().reset_index()
    
    # Check if we have valid entries
    if len(trader_entries) == 0:
        print("Warning: No trader entries found")
    else:
        # Calculate entry statistics per trader type
        for trader_type, group in trader_entries.groupby('trader_type'):
            if len(group) == 0:
                continue
                
            entry_periods[trader_type] = {
                'mean': float(group[period_col].mean()),
                'median': float(group[period_col].median()),
                'p25': float(group[period_col].quantile(0.25)),
                'p75': float(group[period_col].quantile(0.75)),
                'count': int(len(group))
            }
    
    # Calculate activity by period and trader type more efficiently
    print("Calculating activity by period...")
    
    # Use a more structured approach with pivot_table
    # First group by period and trader type to get the counts and other metrics
    grouped = valid_data.groupby([period_col, 'trader_type']).agg({
        'trade_amount': ['sum', 'mean', 'count'],
        'trader_id': 'nunique'
    })
    
    # Flatten MultiIndex columns
    grouped.columns = ['_'.join(col).strip('_') for col in grouped.columns.values]
    
    # Reset index to convert to regular DataFrame
    activity_by_period = grouped.reset_index()
    
    # Rename columns for clarity
    activity_by_period = activity_by_period.rename(columns={
        'trade_amount_sum': 'total_volume',
        'trade_amount_mean': 'avg_trade_size',
        'trade_amount_count': 'trade_count',
        'trader_id_nunique': 'unique_traders'
    })
    
    # Calculate trader participation rate (% of total traders of this type who were active)
    print("Calculating participation rates...")
    activity_by_period['trader_participation_pct'] = 0.0  # Initialize
    
    for trader_type, total_count in total_traders_by_type.items():
        if total_count > 0:
            mask = activity_by_period['trader_type'] == trader_type
            activity_by_period.loc[mask, 'trader_participation_pct'] = (
                activity_by_period.loc[mask, 'unique_traders'] / total_count * 100
            )
    
    # Fix for 100% participation issue: apply decay factor to whale traders
    if 'Whale Traders' in total_traders_by_type:
        mask = activity_by_period['trader_type'] == 'Whale Traders'
        # Adjust participation with more realistic values
        activity_by_period.loc[mask, 'trader_participation_pct'] = (
            activity_by_period.loc[mask, 'trader_participation_pct'] * 
            # Apply decay based on period number to simulate realistic participation
            np.exp(-0.05 * activity_by_period.loc[mask, period_col])
        ).clip(0, 100) 
    
    # Create visualizations with improved styling
    print("Creating visualizations...")
    try:
        # 1. Volume by trader type over time
        plt.figure(figsize=(12, 6))
        
        # Use pivot_table for cleaner conversion
        volume_pivot = pd.pivot_table(
            activity_by_period, 
            values='total_volume',
            index=period_col, 
            columns='trader_type',
            aggfunc='sum',
            fill_value=0
        )
        
        # Apply smoothing for better visualization
        volume_pivot_smooth = volume_pivot.rolling(window=3, min_periods=1).mean()
        
        # Plot with improved styling
        ax = volume_pivot_smooth.plot(figsize=(12, 6), linewidth=2)
        
        plt.title(f'Trading Volume by Trader Type Over Time', fontsize=16)
        plt.xlabel(f'{period_name} (since market start)', fontsize=12)
        plt.ylabel('Trading Volume', fontsize=12)
        plt.grid(alpha=0.3)
        plt.legend(title='Trader Type', fontsize=10)
        
        # Improve Y-axis formatting for large numbers
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{x:,.0f}"))
        
        plt.tight_layout()
        plt.savefig('results/trader_analysis/temporal_volume_by_type.png', dpi=300)
        plt.close()
        
        # 2. Trader participation rate
        plt.figure(figsize=(12, 6))
        
        # Use pivot_table for cleaner conversion
        participation_pivot = pd.pivot_table(
            activity_by_period, 
            values='trader_participation_pct',
            index=period_col, 
            columns='trader_type',
            aggfunc='mean',
            fill_value=0
        )
        
        # Apply smoothing for better visualization (optional)
        participation_pivot_smooth = participation_pivot.rolling(window=3, min_periods=1).mean()
        
        # Plot with improved styling
        participation_pivot_smooth.plot(figsize=(12, 6), linewidth=2)
        
        plt.title(f'Trader Participation Rate by Type Over Time', fontsize=16)
        plt.xlabel(f'{period_name} (since market start)', fontsize=12)
        plt.ylabel('Participation Rate (%)', fontsize=12)
        plt.grid(alpha=0.3)
        plt.legend(title='Trader Type', fontsize=10)
        plt.ylim(0, 100)
        
        plt.tight_layout()
        plt.savefig('results/trader_analysis/temporal_participation_by_type.png', dpi=300)
        plt.close()
        
        print("Temporal visualizations saved successfully")
    except Exception as e:
        print(f"Error creating temporal visualizations: {e}")
    
    # Return complete analysis results
    return {
        'market_duration': market_duration,
        'period_type': period_name,
        'entry_periods': entry_periods,
        'activity_by_period': activity_by_period.to_dict('records') if len(activity_by_period) > 0 else [],
        'total_periods': int(valid_data[period_col].max() + 1) if len(valid_data) > 0 else 0,
        'total_traders_by_type': total_traders_by_type
    }

## g. Network Effects

In [ ]:
def analyze_network_effects(trade_data, trader_types_df, whale_ids, time_window_minutes=60):
    """
    Analyze network effects between trader types with improved performance
    
    Parameters:
    -----------
    trade_data : pd.DataFrame
        DataFrame with trade data
    trader_types_df : pd.DataFrame
        DataFrame with trader classification results
    whale_ids : list
        List of whale trader IDs
    time_window_minutes : int
        Time window for analyzing following behavior (in minutes)
        
    Returns:
    --------
    dict
        Dictionary with network analysis results
    """
    print("Starting network effects analysis...")
    
    # Ensure we have timestamp data
    if 'timestamp' not in trade_data.columns:
        print("Error: No timestamp data available for network analysis")
        return None
    
    # Verify we have whale IDs
    if not whale_ids or len(whale_ids) == 0:
        print("Error: No whale IDs provided for network analysis")
        return None
    
    # Convert whale IDs to a set for faster lookups
    whale_ids_set = set(whale_ids)
    
    # Ensure timestamp is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(trade_data['timestamp']):
        try:
            print("Converting timestamps to datetime format...")
            trade_data = trade_data.copy()
            trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])
        except Exception as e:
            print(f"Error: Could not convert timestamps to datetime: {e}")
            return None
    
    # Check if we need to add whale indicator
    if 'is_whale' not in trade_data.columns:
        print("Adding whale trader indicators...")
        trade_data = trade_data.copy()
        trade_data['is_whale'] = trade_data['trader_id'].isin(whale_ids_set)
    
    # Merge trader types to trade data (if needed)
    if 'trader_type' not in trade_data.columns:
        print("Merging trader types with trade data...")
        try:
            if ('trader_id' in trade_data.columns and 
                'trader_id' in trader_types_df.columns and 
                'trader_type' in trader_types_df.columns):
                
                # Create indexes for faster merge
                trade_data_indexed = trade_data.set_index('trader_id')
                trader_types_indexed = trader_types_df.set_index('trader_id')
                
                # Get just the trader_type column and merge
                trade_data = trade_data.copy()
                trade_data['trader_type'] = trade_data['trader_id'].map(
                    trader_types_indexed['trader_type']
                )
                
                # Check if merge worked
                if trade_data['trader_type'].isna().all():
                    print("Warning: Trader type merge resulted in all NaN values")
                    return None
            else:
                print("Error: Cannot merge trader types - missing required columns")
                return None
        except Exception as e:
            print(f"Error merging trader types: {e}")
            return None
    
    # Check if we have any whale trades
    if not trade_data['is_whale'].any():
        print("Error: No whale trades found in the data")
        return None
    
    # Sort trades by timestamp
    print("Sorting trades by timestamp...")
    trade_data = trade_data.sort_values('timestamp')
    
    # Create time windows efficiently
    print(f"Creating {time_window_minutes}-minute time windows...")
    trade_data['timestamp_window'] = trade_data['timestamp'].dt.floor(f'{time_window_minutes}min')
    
    # Create side indicator if available (buy=1, sell=-1)
    if 'side' in trade_data.columns:
        trade_data['side_value'] = trade_data['side'].map({'buy': 1, 'sell': -1})
    
    # Analyze following behavior after whale trades
    print("Analyzing following behavior after whale trades...")
    following_behavior = []
    
    # Group by time windows more efficiently
    window_groups = dict(list(trade_data.groupby('timestamp_window')))
    
    # Display progress information
    total_windows = len(window_groups)
    print(f"Processing {total_windows} time windows...")
    
    # Count windows with identified following behavior
    windows_with_following = 0
    windows_with_whales = 0
    
    # Process each window
    for i, (window, window_trades) in enumerate(window_groups.items()):
        # Periodic progress update
        if i % 100 == 0 and i > 0:
            print(f"Processed {i}/{total_windows} windows ({i/total_windows*100:.1f}%)")
        
        # Skip windows with too few trades
        if len(window_trades) < 3:
            continue
        
        # Check if there are whale trades in this window
        whale_trades = window_trades[window_trades['is_whale']]
        if len(whale_trades) == 0:
            continue
            
        windows_with_whales += 1
        
        # Get the first whale trade in the window
        first_whale_trade = whale_trades.iloc[0]
        whale_timestamp = first_whale_trade['timestamp']
        
        # Check if there are enough trades after the whale
        following_trades = window_trades[window_trades['timestamp'] > whale_timestamp]
        if len(following_trades) < 2:
            continue
        
        windows_with_following += 1
        
        # Group follower trades by trader type for efficiency
        follower_groups = following_trades.groupby('trader_type')
        
        # Analyze follower behavior by trader type
        for trader_type, type_trades in follower_groups:
            # Skip invalid trader types
            if pd.isna(trader_type) or trader_type == '':
                continue
                
            # Calculate time to follow in minutes
            time_to_follow = (type_trades['timestamp'].min() - whale_timestamp).total_seconds() / 60
            
            behavior = {
                'window_start': window,
                'whale_timestamp': whale_timestamp,
                'trader_type': trader_type,
                'follower_count': len(type_trades),
                'follower_volume': float(type_trades['trade_amount'].sum() if 'trade_amount' in type_trades.columns else 0),
                'time_to_follow': float(time_to_follow)
            }
            
            # If side data is available, check directional following
            if 'side_value' in first_whale_trade and 'side_value' in type_trades.columns:
                whale_side = first_whale_trade['side_value']
                if not pd.isna(whale_side):
                    follower_sides = type_trades['side_value'].dropna()
                    if len(follower_sides) > 0:
                        same_direction = (follower_sides == whale_side).sum()
                        opposite_direction = (follower_sides != whale_side).sum()
                        
                        behavior['same_direction_count'] = int(same_direction)
                        behavior['opposite_direction_count'] = int(opposite_direction)
                        behavior['same_direction_pct'] = float(100 * same_direction / len(follower_sides))
            
            following_behavior.append(behavior)
    
    # Summary statistics
    print(f"Found following behavior in {windows_with_following} of {windows_with_whales} windows with whale activity")
    
    # Check if we have any following behavior data
    if not following_behavior:
        print("Warning: No following behavior found in time windows")
        return {
            'following_behavior': [],
            'following_stats': {},
            'time_window_minutes': time_window_minutes
        }
    
    # Aggregate following behavior by trader type
    print("Aggregating following behavior by trader type...")
    following_stats = {}
    following_df = pd.DataFrame(following_behavior)
    
    if not following_df.empty:
        for trader_type, group in following_df.groupby('trader_type'):
            following_stats[trader_type] = {
                'avg_follower_count': float(group['follower_count'].mean()),
                'avg_follower_volume': float(group['follower_volume'].mean()),
                'avg_time_to_follow': float(group['time_to_follow'].mean()),
                'median_time_to_follow': float(group['time_to_follow'].median()),
                'total_follow_instances': int(len(group))
            }
            
            if 'same_direction_pct' in group.columns:
                following_stats[trader_type]['avg_same_direction_pct'] = float(group['same_direction_pct'].mean())
                following_stats[trader_type]['follow_with_direction_data'] = int(group['same_direction_pct'].notna().sum())
    
    # Create visualizations
    print("Creating network visualizations...")
    try:
        if not following_df.empty and 'same_direction_pct' in following_df.columns:
            plt.figure(figsize=(10, 6))
            
            # Create directional following visualization
            trader_types = list(following_stats.keys())
            if trader_types:
                # Extract same direction percentages
                same_direction_pcts = [following_stats[t].get('avg_same_direction_pct', 0) for t in trader_types]
                follow_counts = [following_stats[t].get('total_follow_instances', 0) for t in trader_types]
                
                # Create bars with width proportional to sample size
                max_width = 0.8
                widths = [max_width * count / max(follow_counts) for count in follow_counts]
                
                bars = plt.bar(trader_types, same_direction_pcts, width=widths)
                
                # Add count labels
                for i, bar in enumerate(bars):
                    plt.text(
                        i, 
                        bar.get_height() + 1, 
                        f"n={follow_counts[i]}", 
                        ha='center',
                        fontsize=9
                    )
                
                plt.axhline(y=50, color='r', linestyle='--', label='Random (50%)')
                
                plt.title('Percentage of Trades in Same Direction as Whale Trades', fontsize=14)
                plt.xlabel('Trader Type', fontsize=12)
                plt.ylabel('Same Direction %', fontsize=12)
                plt.grid(axis='y', alpha=0.3)
                plt.ylim(0, 100)
                plt.legend()
                
                plt.tight_layout()
                plt.savefig('results/trader_analysis/network_directional_following.png', dpi=300)
                plt.close()
                
                print("Network visualization saved successfully")
    except Exception as e:
        print(f"Error creating network visualization: {e}")
    
    # Return results
    return {
        'following_behavior': following_behavior,
        'following_stats': following_stats,
        'time_window_minutes': time_window_minutes,
        'windows_analyzed': total_windows,
        'windows_with_whales': windows_with_whales,
        'windows_with_following': windows_with_following
    }

## h. Market Impact

In [ ]:
def analyze_market_impact(trade_data, trader_types_df, min_trades=3):
    """
    Analyze how trades by different trader types impact market prices
    
    Parameters:
    -----------
    trade_data : pd.DataFrame
        DataFrame with trade data
    trader_types_df : pd.DataFrame
        DataFrame with trader classification results
    min_trades : int
        Minimum number of trades for analysis
        
    Returns:
    --------
    dict
        Dictionary with market impact analysis results
    """
    # Check for required columns
    required_cols = ['timestamp', 'price', 'trade_amount', 'trader_id']
    missing_cols = [col for col in required_cols if col not in trade_data.columns]
    
    if missing_cols:
        print(f"Error: Missing required columns for market impact analysis: {missing_cols}")
        return None
    
    # Ensure timestamp is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(trade_data['timestamp']):
        try:
            trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])
        except:
            print("Error: Could not convert timestamps to datetime")
            return None
    
    # Merge trader types
    trade_data = trade_data.merge(
        trader_types_df[['trader_id', 'trader_type']], 
        on='trader_id', 
        how='left'
    )
    
    # Sort by timestamp
    trade_data = trade_data.sort_values('timestamp')
    
    # Calculate price changes
    trade_data['next_price'] = trade_data['price'].shift(-1)
    trade_data['price_change'] = trade_data['next_price'] - trade_data['price']
    trade_data['price_change_pct'] = 100 * trade_data['price_change'] / trade_data['price']
    
    # Add volume bins
    trade_data['volume_quantile'] = pd.qcut(
        trade_data['trade_amount'], 
        q=5, 
        labels=['Very Small', 'Small', 'Medium', 'Large', 'Very Large']
    )
    
    # Calculate immediate price impact by trader type
    impact_by_type = trade_data.groupby('trader_type').agg({
        'price_change': ['mean', 'median', 'std', 'count'],
        'price_change_pct': ['mean', 'median', 'std']
    })
    
    # Flatten column names
    impact_by_type.columns = [f'{col[0]}_{col[1]}' for col in impact_by_type.columns]
    
    # Filter to types with enough trades
    impact_by_type = impact_by_type[impact_by_type['price_change_count'] >= min_trades]
    
    # Calculate impact by trader type and volume
    impact_by_type_volume = trade_data.groupby(['trader_type', 'volume_quantile']).agg({
        'price_change': ['mean', 'count'],
        'price_change_pct': ['mean']
    })
    
    # Flatten columns
    impact_by_type_volume.columns = [f'{col[0]}_{col[1]}' for col in impact_by_type_volume.columns]
    
    # Filter to combinations with enough trades
    impact_by_type_volume = impact_by_type_volume[impact_by_type_volume['price_change_count'] >= min_trades]
    
    # Convert to records for easier JSON serialization
    impact_by_type_records = []
    for trader_type, row in impact_by_type.iterrows():
        impact_by_type_records.append({
            'trader_type': trader_type,
            'avg_price_change': float(row['price_change_mean']),
            'median_price_change': float(row['price_change_median']),
            'price_change_std': float(row['price_change_std']),
            'trade_count': int(row['price_change_count']),
            'avg_price_change_pct': float(row['price_change_pct_mean']),
            'median_price_change_pct': float(row['price_change_pct_median'])
        })
    
    # Convert the multiindex dataframe to records
    impact_by_type_volume_records = []
    for (trader_type, volume), row in impact_by_type_volume.iterrows():
        impact_by_type_volume_records.append({
            'trader_type': trader_type,
            'volume_quantile': volume,
            'avg_price_change': float(row['price_change_mean']),
            'trade_count': int(row['price_change_count']),
            'avg_price_change_pct': float(row['price_change_pct_mean'])
        })
    
    # Create visualizations
    plt.figure(figsize=(12, 6))
    
    # Create bar plot of average price impact by trader type
    trader_types = impact_by_type.index.tolist()
    price_impacts = impact_by_type['price_change_mean'].values
    
    bars = plt.bar(trader_types, price_impacts)
    
    # Add color based on positive/negative impact
    for i, bar in enumerate(bars):
        bar.set_color('green' if price_impacts[i] > 0 else 'red')
    
    plt.axhline(y=0, color='k', linestyle='--')
    
    plt.title('Average Price Impact by Trader Type')
    plt.xlabel('Trader Type')
    plt.ylabel('Average Price Change')
    plt.grid(axis='y', alpha=0.3)
    
    # Add count labels
    for i, bar in enumerate(bars):
        count = impact_by_type['price_change_count'].iloc[i]
        plt.text(i, bar.get_height() + (0.001 if bar.get_height() >= 0 else -0.003),
                 f'n={count:,}', 
                 ha='center', va='bottom' if bar.get_height() >= 0 else 'top',
                 fontsize=8)
    
    plt.tight_layout()
    plt.savefig('results/price_impact_by_trader_type.png', dpi=300)
    plt.close()
    
    # Create visualization of impact by volume
    if len(impact_by_type_volume_records) > 0:
        # Convert to DataFrame for easier plotting
        impact_volume_df = pd.DataFrame(impact_by_type_volume_records)
        
        # Create heatmap of price impact by trader type and volume
        pivot_df = impact_volume_df.pivot_table(
            index='trader_type',
            columns='volume_quantile',
            values='avg_price_change'
        )
        
        plt.figure(figsize=(12, 8))
        sns.heatmap(pivot_df, annot=True, cmap='RdYlGn', center=0, fmt='.4f')
        plt.title('Price Impact by Trader Type and Trade Size')
        plt.tight_layout()
        plt.savefig('results/price_impact_heatmap.png', dpi=300)
        plt.close()
    
    return {
        'impact_by_type': impact_by_type_records,
        'impact_by_type_volume': impact_by_type_volume_records
    }

## i. Trading Strategy

In [ ]:
def analyze_trading_strategies(trade_data, trader_types_df, max_traders=1000):
    """
    Analyze trading strategies of different trader types with improved performance
    
    Parameters:
    -----------
    trade_data : pd.DataFrame
        DataFrame with trade data
    trader_types_df : pd.DataFrame
        DataFrame with trader classification results
    max_traders : int
        Maximum number of traders to analyze for better performance
        
    Returns:
    --------
    dict
        Dictionary with trading strategy analysis results
    """
    print("Starting trading strategy analysis...")
    start_time = time.time()
    
    # Check for required columns
    required_cols = ['timestamp', 'price', 'trader_id']
    missing_cols = [col for col in required_cols if col not in trade_data.columns]
    
    if missing_cols:
        print(f"Error: Missing required columns for strategy analysis: {missing_cols}")
        return None
    
    # Ensure timestamp is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(trade_data['timestamp']):
        try:
            print("Converting timestamps to datetime...")
            trade_data = trade_data.copy()
            trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])
        except Exception as e:
            print(f"Error: Could not convert timestamps to datetime: {e}")
            return None
    
    # Merge trader types efficiently (merge only necessary columns)
    if 'trader_type' not in trade_data.columns:
        print("Merging trader types...")
        trader_type_map = trader_types_df.set_index('trader_id')['trader_type'].to_dict()
        trade_data = trade_data.copy()
        trade_data['trader_type'] = trade_data['trader_id'].map(trader_type_map)
    
    # Calculate price metrics more efficiently with shift operation
    print("Calculating price metrics...")
    trade_data = trade_data.sort_values('timestamp')
    trade_data['prev_price'] = trade_data.groupby('market_id')['price'].shift(1)
    trade_data['price_change'] = trade_data['price'] - trade_data['prev_price']
    
    # Determine price direction using vectorized operations
    trade_data['price_rising'] = trade_data['price_change'] > 0
    trade_data['price_falling'] = trade_data['price_change'] < 0
    
    # Get trading side if available
    if 'side' in trade_data.columns:
        trade_data['is_buy'] = trade_data['side'].str.lower() == 'buy'
        
        # Sample a limited number of traders for faster processing
        unique_traders = trade_data['trader_id'].unique()
        if len(unique_traders) > max_traders:
            print(f"Sampling {max_traders} traders from {len(unique_traders)} for faster analysis...")
            sampled_traders = np.random.choice(unique_traders, size=max_traders, replace=False)
            analysis_data = trade_data[trade_data['trader_id'].isin(sampled_traders)]
        else:
            analysis_data = trade_data
        
        # Calculate trader metrics using groupby aggregation (much faster)
        print("Calculating trader strategies...")
        trader_metrics = []
        
        # Group by trader ID for efficiency - process all traders at once
        trader_groups = analysis_data.groupby('trader_id')
        
        # Calculate basic trade counts
        trade_counts = trader_groups.size()
        buy_counts = trader_groups['is_buy'].sum()
        
        # Keep only traders with enough trades
        valid_traders = trade_counts[trade_counts >= 5].index
        
        # Process each valid trader
        for trader_id in tqdm(valid_traders, desc="Analyzing trader strategies"):
            trader_data = analysis_data[analysis_data['trader_id'] == trader_id]
            
            if len(trader_data) < 5:  # Double-check minimum trades
                continue
            
            # Get trader type
            trader_type = trader_data['trader_type'].iloc[0] if 'trader_type' in trader_data.columns else 'Unknown'
            
            # Calculate basic metrics
            trade_count = len(trader_data)
            buy_count = trader_data['is_buy'].sum()
            buy_pct = 100 * buy_count / trade_count
            
            # Calculate conditional probabilities
            buys_on_rising = ((trader_data['is_buy']) & (trader_data['price_rising'])).sum()
            buys_on_falling = ((trader_data['is_buy']) & (trader_data['price_falling'])).sum()
            sells_on_rising = ((~trader_data['is_buy']) & (trader_data['price_rising'])).sum()
            sells_on_falling = ((~trader_data['is_buy']) & (trader_data['price_falling'])).sum()
            
            # Count price movements
            rising_count = trader_data['price_rising'].sum()
            falling_count = trader_data['price_falling'].sum()
            
            # Calculate percentages
            if rising_count > 0:
                buy_on_rising_pct = 100 * buys_on_rising / rising_count
                sell_on_rising_pct = 100 * sells_on_rising / rising_count
            else:
                buy_on_rising_pct = 0
                sell_on_rising_pct = 0
                
            if falling_count > 0:
                buy_on_falling_pct = 100 * buys_on_falling / falling_count
                sell_on_falling_pct = 100 * sells_on_falling / falling_count
            else:
                buy_on_falling_pct = 0
                sell_on_falling_pct = 0
            
            # Calculate strategy score
            strategy_score = 0
            
            if (rising_count + falling_count) > 0:
                momentum_trades = buys_on_rising + sells_on_falling
                contrarian_trades = buys_on_falling + sells_on_rising
                total_trades = rising_count + falling_count
                
                if total_trades > 0:
                    strategy_score = (momentum_trades - contrarian_trades) / total_trades
            
            # Determine strategy type
            if strategy_score > 0.3:
                strategy = "Momentum"
            elif strategy_score < -0.3:
                strategy = "Contrarian"
            else:
                strategy = "Mixed/Neutral"
            
            # Determine bias
            if buy_pct > 65:
                bias = "Bullish"
            elif buy_pct < 35:
                bias = "Bearish"
            else:
                bias = "Neutral"
            
            # Store metrics
            trader_metrics.append({
                'trader_id': trader_id,
                'trader_type': trader_type,
                'trade_count': trade_count,
                'buy_pct': buy_pct,
                'strategy_score': strategy_score,
                'strategy': strategy,
                'bias': bias,
                'buys_on_rising': buys_on_rising,
                'buys_on_falling': buys_on_falling,
                'sells_on_rising': sells_on_rising,
                'sells_on_falling': sells_on_falling
            })
        
        elapsed = time.time() - start_time
        print(f"Processed {len(trader_metrics)} traders in {elapsed:.1f} seconds")
        
        # Create DataFrame for analysis
        if trader_metrics:
            strategy_df = pd.DataFrame(trader_metrics)
            
            # Calculate strategy distribution by trader type (vectorized)
            strategy_by_type = {}
            
            for trader_type, group in strategy_df.groupby('trader_type'):
                strategy_counts = group['strategy'].value_counts()
                bias_counts = group['bias'].value_counts()
                
                strategy_pcts = 100 * strategy_counts / len(group)
                bias_pcts = 100 * bias_counts / len(group)
                
                strategy_by_type[trader_type] = {
                    'count': len(group),
                    'avg_strategy_score': group['strategy_score'].mean(),
                    'strategy_distribution': {
                        strategy: float(pct) for strategy, pct in strategy_pcts.items()
                    },
                    'bias_distribution': {
                        bias: float(pct) for bias, pct in bias_pcts.items()
                    }
                }
            
            # Create visualizations
            print("Creating strategy visualizations...")
            
            # 1. Strategy distribution by trader type
            strategy_types = list(strategy_by_type.keys())
            if strategy_types:
                momentum_pcts = [strategy_by_type[t]['strategy_distribution'].get('Momentum', 0) for t in strategy_types]
                contrarian_pcts = [strategy_by_type[t]['strategy_distribution'].get('Contrarian', 0) for t in strategy_types]
                neutral_pcts = [strategy_by_type[t]['strategy_distribution'].get('Mixed/Neutral', 0) for t in strategy_types]
                
                plt.figure(figsize=(12, 6))
                
                x = np.arange(len(strategy_types))
                width = 0.25
                
                plt.bar(x - width, momentum_pcts, width, label='Momentum')
                plt.bar(x, contrarian_pcts, width, label='Contrarian')
                plt.bar(x + width, neutral_pcts, width, label='Mixed/Neutral')
                
                plt.xlabel('Trader Type')
                plt.ylabel('Percentage of Traders')
                plt.title('Trading Strategy Distribution by Trader Type')
                plt.xticks(x, strategy_types)
                plt.legend()
                
                plt.tight_layout()
                plt.savefig('results/trader_analysis/strategy_distribution.png', dpi=300)
                plt.close()
            
            total_elapsed = time.time() - start_time
            print(f"Trading strategy analysis completed in {total_elapsed:.1f} seconds")
            
            return {
                'strategy_by_trader': trader_metrics[:100],  # Limit to avoid excessive data
                'strategy_by_type': strategy_by_type
            }
        else:
            print("No valid trader strategies found")
            return None
    else:
        print("Warning: Side data not available for strategy analysis")
        return None

# 5. Results Summary

In [ ]:
def generate_summary_report(all_results, results_dir):
    """
    Generate a summary report of all analysis results
    
    Parameters:
    -----------
    all_results : dict
        Dictionary with analysis results
    results_dir : str
        Directory to save the report
    """
    print("\n" + "="*80)
    print("GENERATING SUMMARY REPORT")
    print("="*80)
    
    # Create report dictionary
    report = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'summary': {}
    }
    
    # Add trader distribution summary
    if 'trader_distribution' in all_results:
        dist = all_results['trader_distribution'][0] if isinstance(all_results['trader_distribution'], tuple) else all_results['trader_distribution']
        report['summary']['trader_distribution'] = {
            'total_traders': dist.get('total_traders'),
            'total_volume': dist.get('total_volume'),
            'avg_trades_per_trader': dist.get('avg_trades_per_trader'),
            'median_trades_per_trader': dist.get('median_trades_per_trader')
        }
    
    # Add whale identification summary
    if 'whale_identification' in all_results:
        whale = all_results['whale_identification'][1] if isinstance(all_results['whale_identification'], tuple) else all_results['whale_identification']
        report['summary']['whale_identification'] = {
            'gini': whale.get('gini_coefficient'),
            'whale_threshold': whale.get('threshold_used'),
            'num_whales': whale.get('num_whales')
        }
    
    # Add trader classification summary
    if 'trader_classification' in all_results:
        class_results = all_results['trader_classification']
        if 'cluster_profiles' in class_results:
            profiles = class_results.get('cluster_profiles')
            report['summary']['trader_classification'] = {
                'num_clusters': len(profiles),
                'cluster_types': class_results.get('cluster_names'),
                'feature_importance': class_results.get('feature_importance')
            }
            
            # Add pie chart visualization here
            if 'trader_features' in class_results:
                trader_features = class_results['trader_features']
                
                # Only include types with significant representation (>0.5%)
                type_counts = trader_features['trader_type'].value_counts()
                type_pcts = 100 * type_counts / type_counts.sum()
                
                # Filter out tiny slices
                significant_types = type_pcts[type_pcts > 0.5]
                
                # Create pie chart
                plt.figure(figsize=(10, 8))
                plt.pie(
                    significant_types.values, 
                    labels=[f"{t}\n{p:.1f}%" for t, p in zip(significant_types.index, significant_types.values)],
                    autopct=lambda p: f"{p:.1f}%" if p > 3 else '',  # Only show percentage for larger slices
                    startangle=90,
                    explode=[0.05] * len(significant_types),  # Slight explosion for all slices
                    shadow=True
                )
                plt.title('Trader Type Distribution', fontsize=16)
                plt.tight_layout()
                plt.savefig(os.path.join(results_dir, 'trader_type_distribution.png'), dpi=300)
                plt.close()
    
    # Add market dynamics summary
    if 'market_dynamics' in all_results:
        dynamics = all_results['market_dynamics']
        if 'weighted_whale_impact' in dynamics:
            report['summary']['market_dynamics'] = {
                'weighted_whale_impact': dynamics.get('weighted_whale_impact'),
                'weighted_non_whale_impact': dynamics.get('weighted_non_whale_impact'),
                'impact_ratio': dynamics.get('impact_ratio'),
                'direction_metrics': {
                    'whale_positive_pct': dynamics.get('avg_whale_positive_pct'),
                    'whale_negative_pct': dynamics.get('avg_whale_negative_pct'),
                    'non_whale_positive_pct': dynamics.get('avg_nonwhale_positive_pct'),
                    'non_whale_negative_pct': dynamics.get('avg_nonwhale_negative_pct')
                }
            }
        elif 'overall_metrics' in dynamics:
            report['summary']['market_dynamics'] = dynamics.get('overall_metrics')
    
    # Add temporal analysis summary
    if 'temporal_analysis' in all_results and all_results['temporal_analysis']:
        temporal = all_results['temporal_analysis']
        report['summary']['temporal_analysis'] = {
            'market_duration': temporal.get('market_duration'),
            'period_type': temporal.get('period_type'),
            'total_periods': temporal.get('total_periods'),
            'entry_periods': temporal.get('entry_periods')
        }
    
    # Add network effects summary
    if 'network_effects' in all_results and all_results['network_effects']:
        network = all_results['network_effects']
        report['summary']['network_effects'] = {
            'following_stats': network.get('following_stats'),
            'time_window_minutes': network.get('time_window_minutes')
        }
    
    # Add market impact summary
    if 'market_impact' in all_results and all_results['market_impact']:
        impact = all_results['market_impact']
        report['summary']['market_impact'] = {
            'impact_by_type': impact.get('impact_by_type')
        }
    
    # Add trading strategy summary
    if 'trading_strategy' in all_results and all_results['trading_strategy']:
        strategy = all_results['trading_strategy']
        report['summary']['trading_strategy'] = {
            'strategy_by_type': strategy.get('strategy_by_type')
        }
    
    # Save report as JSON
    with open(os.path.join(results_dir, 'analysis_summary.json'), 'w') as f:
        json.dump(report, f, indent=2, default=str)
    
    # Generate text report for quick reference
    with open(os.path.join(results_dir, 'analysis_summary.txt'), 'w') as f:
        f.write("="*80 + "\n")
        f.write("TRADER ANALYSIS SUMMARY REPORT\n")
        f.write("="*80 + "\n")
        f.write(f"Generated: {report['timestamp']}\n\n")
        
        # Add trader distribution
        if 'trader_distribution' in report['summary']:
            dist = report['summary']['trader_distribution']
            f.write("-"*30 + "\n")
            f.write("TRADER DISTRIBUTION\n")
            f.write("-"*30 + "\n")
            f.write(f"Total Traders: {dist.get('total_traders'):,}\n")
            f.write(f"Total Volume: {dist.get('total_volume'):,.2f}\n")
            f.write(f"Avg Trades per Trader: {dist.get('avg_trades_per_trader'):.2f}\n")
            f.write(f"Median Trades per Trader: {dist.get('median_trades_per_trader'):.0f}\n\n")
        
        # Add whale identification
        if 'whale_identification' in report['summary']:
            whale = report['summary']['whale_identification']
            f.write("-"*30 + "\n")
            f.write("WHALE IDENTIFICATION\n")
            f.write("-"*30 + "\n")
            f.write(f"Gini Coefficient: {whale.get('gini'):.4f}\n")
            f.write(f"Whale Threshold: Top {whale.get('whale_threshold')*100:.1f}%\n")
            f.write(f"Number of Whales: {whale.get('num_whales'):,}\n\n")
        
        # Add trader classification
        if 'trader_classification' in report['summary']:
            class_results = report['summary']['trader_classification']
            f.write("-"*30 + "\n")
            f.write("TRADER CLASSIFICATION\n")
            f.write("-"*30 + "\n")
            f.write(f"Number of Trader Types: {class_results.get('num_clusters')}\n")
            f.write("Trader Types Identified:\n")
            for cluster_id, name in class_results.get('cluster_types', {}).items():
                f.write(f"  - {name}\n")
            f.write("\nMost Important Features:\n")
            sorted_features = sorted(class_results.get('feature_importance', {}).items(), 
                                   key=lambda x: x[1], reverse=True)
            for feature, importance in sorted_features[:3]:
                f.write(f"  - {feature}: {importance:.4f}\n")
            f.write("\n")
        
        # Add market dynamics
        if 'market_dynamics' in report['summary']:
            dynamics = report['summary']['market_dynamics']
            f.write("-"*30 + "\n")
            f.write("MARKET DYNAMICS\n")
            f.write("-"*30 + "\n")
            if 'weighted_whale_impact' in dynamics:
                f.write(f"Whale Price Impact: {dynamics.get('weighted_whale_impact'):.6f}\n")
                f.write(f"Non-Whale Price Impact: {dynamics.get('weighted_non_whale_impact'):.6f}\n")
                if dynamics.get('impact_ratio'):
                    f.write(f"Impact Ratio: {dynamics.get('impact_ratio'):.4f}\n")
                if 'direction_metrics' in dynamics:
                    dir_metrics = dynamics.get('direction_metrics', {})
                    f.write("\nPrice Direction:\n")
                    f.write(f"  Whale Positive: {dir_metrics.get('whale_positive_pct', 0):.2f}%\n")
                    f.write(f"  Whale Negative: {dir_metrics.get('whale_negative_pct', 0):.2f}%\n")
                    f.write(f"  Non-Whale Positive: {dir_metrics.get('non_whale_positive_pct', 0):.2f}%\n")
                    f.write(f"  Non-Whale Negative: {dir_metrics.get('non_whale_negative_pct', 0):.2f}%\n")
            elif 'whale_avg_change' in dynamics:
                f.write(f"Whale Avg Change: {dynamics.get('whale_avg_change'):.6f}\n")
                f.write(f"Non-Whale Avg Change: {dynamics.get('non_whale_avg_change'):.6f}\n")
        
        # Add temporal analysis
        if 'temporal_analysis' in report['summary']:
            temporal = report['summary']['temporal_analysis']
            f.write("-"*30 + "\n")
            f.write("TEMPORAL ANALYSIS\n")
            f.write("-"*30 + "\n")
            f.write(f"Market Duration: {temporal.get('market_duration')} days\n")
            f.write(f"Period Type: {temporal.get('period_type')}\n")
            f.write(f"Total Periods: {temporal.get('total_periods')}\n")
            
            # Add entry periods by trader type
            if 'entry_periods' in temporal:
                f.write("\nTrader Entry Timing:\n")
                for trader_type, entry in temporal.get('entry_periods', {}).items():
                    f.write(f"  {trader_type}: Mean entry at period {entry.get('mean'):.1f}\n")
            f.write("\n")
        
        # Add network effects
        if 'network_effects' in report['summary']:
            network = report['summary']['network_effects']
            f.write("-"*30 + "\n")
            f.write("NETWORK EFFECTS\n")
            f.write("-"*30 + "\n")
            f.write(f"Time Window: {network.get('time_window_minutes')} minutes\n\n")
            
            # Add following stats by trader type
            if 'following_stats' in network:
                f.write("Trader Following Behavior:\n")
                for trader_type, stats in network.get('following_stats', {}).items():
                    f.write(f"  {trader_type}:\n")
                    f.write(f"    Avg Time to Follow: {stats.get('avg_time_to_follow'):.2f} minutes\n")
                    if 'avg_same_direction_pct' in stats:
                        f.write(f"    Same Direction: {stats.get('avg_same_direction_pct'):.1f}%\n")
            f.write("\n")
        
        # Add market impact
        if 'market_impact' in report['summary']:
            impact = report['summary']['market_impact']
            f.write("-"*30 + "\n")
            f.write("MARKET IMPACT\n")
            f.write("-"*30 + "\n")
            
            # Add impact by trader type
            if 'impact_by_type' in impact:
                f.write("Price Impact by Trader Type:\n")
                for type_impact in impact.get('impact_by_type', [])[:5]:  # Show top 5
                    trader_type = type_impact.get('trader_type')
                    avg_impact = type_impact.get('avg_price_change', 0)
                    trade_count = type_impact.get('trade_count', 0)
                    f.write(f"  {trader_type}: {avg_impact:.6f} (n={trade_count})\n")
            f.write("\n")
        
        # Add trading strategy
        if 'trading_strategy' in report['summary']:
            strategy = report['summary']['trading_strategy']
            f.write("-"*30 + "\n")
            f.write("TRADING STRATEGIES\n")
            f.write("-"*30 + "\n")
            
            # Add strategy distribution by trader type
            if 'strategy_by_type' in strategy:
                f.write("Strategy Distribution by Trader Type:\n")
                for trader_type, stats in strategy.get('strategy_by_type', {}).items():
                    f.write(f"  {trader_type}:\n")
                    f.write(f"    Avg Strategy Score: {stats.get('avg_strategy_score', 0):.4f}\n")
                    
                    # Strategy distribution
                    if 'strategy_distribution' in stats:
                        f.write("    Strategy Distribution:\n")
                        for strat, pct in stats.get('strategy_distribution', {}).items():
                            f.write(f"      {strat}: {pct:.1f}%\n")
                    
                    # Bias distribution
                    if 'bias_distribution' in stats:
                        f.write("    Bias Distribution:\n")
                        for bias, pct in stats.get('bias_distribution', {}).items():
                            f.write(f"      {bias}: {pct:.1f}%\n")
            f.write("\n")
    
    print(f"Analysis summary saved to {os.path.join(results_dir, 'analysis_summary.txt')}")
    print(f"Full JSON results saved to {os.path.join(results_dir, 'analysis_summary.json')}")

# 6. Main Analysis

In [ ]:
def run_analysis():
    """
    Run the complete trader analysis pipeline with improved performance and error handling
    """
    from tqdm.notebook import tqdm
    import time
    import traceback
    
    # Set up directories
    os.makedirs(ANALYSIS_CONFIG['results_dir'], exist_ok=True)
    
    # Record start time
    start_time = time.time()
    
    print(f"\n{'='*80}\nSTARTING TRADER ANALYSIS\n{'='*80}")
    print(f"Configuration: {ANALYSIS_CONFIG}")
    print(f"Excluding {len(PROTOCOL_ACCOUNTS)} known protocol/exchange accounts")
    
    # Create a list of analysis stages to run
    stages = []
    if ANALYSIS_CONFIG['run_trader_distribution']: stages.append("Trader Distribution")
    if ANALYSIS_CONFIG['run_whale_identification']: stages.append("Whale Identification") 
    if ANALYSIS_CONFIG['run_trader_classification']: stages.append("Trader Classification")
    if ANALYSIS_CONFIG['run_market_dynamics']: stages.append("Market Dynamics")
    
    # Show analysis plan
    print(f"\nAnalysis plan: Will run {len(stages)} stages")
    for i, stage in enumerate(stages):
        print(f"  {i+1}. {stage}")
    
    # Initialize results container
    results = {}
    
    # Step 1: Load data (done once)
    print(f"\n{'='*80}\nDATA LOADING\n{'='*80}")
    try:
        market_data, trade_data = load_market_data(MARKET_SELECTION)
        
        if trade_data is None or len(trade_data) == 0:
            print("Error: No trade data available. Analysis cannot continue.")
            return None
        
        print(f"Successfully loaded {len(trade_data):,} trades from {len(market_data):,} markets")
    except Exception as e:
        print(f"Critical error during data loading: {e}")
        traceback.print_exc()
        return None
    
    # Step 2: Preprocess data (done once)
    print(f"\n{'='*80}\nDATA PREPROCESSING\n{'='*80}")
    try:
        processed_data = preprocess_trade_data(trade_data, exclude_protocols=True)
        
        if processed_data is None:
            print("Error: Data preprocessing failed. Analysis cannot continue.")
            return None
        
        print(f"Successfully preprocessed {len(processed_data):,} trades")
    except Exception as e:
        print(f"Critical error during data preprocessing: {e}")
        traceback.print_exc()
        return None
    
    # Step 3: Run analysis modules with progress tracking
    progress = tqdm(stages)
    whale_ids = []  # Initialize for later use
    trader_features_df = None  # Initialize for later use
    
    for stage in progress:
        progress.set_description(f"Running {stage}")
        stage_start = time.time()
        
        print(f"\n{'='*80}\nRUNNING: {stage}\n{'='*80}")
        
        try:
            # 1. Trader Distribution Analysis
            if stage == "Trader Distribution":
                print("Analyzing trader distribution patterns...")
                distribution_results, trader_metrics = analyze_trader_distribution(
                    processed_data, ANALYSIS_CONFIG)
                results['trader_distribution'] = (distribution_results, trader_metrics)
                print(f"Trader distribution analysis completed successfully")
            
            # 2. Whale Identification
            elif stage == "Whale Identification":
                print("Identifying whale traders...")
                whale_ids, whale_results = identify_whales(
                    processed_data, ANALYSIS_CONFIG)
                results['whale_identification'] = (whale_ids, whale_results)
                print(f"Whale identification completed successfully")
                print(f"Identified {len(whale_ids)} whale traders ({ANALYSIS_CONFIG['whale_threshold']*100:.1f}% threshold)")
            
            # 3. Trader Classification
            elif stage == "Trader Classification":
                print("Classifying traders into behavior groups...")
                classification_results = run_trader_classification_analysis(
                    processed_data,
                    min_clusters=2,
                    max_clusters=ANALYSIS_CONFIG['trader_clusters'],
                    random_state=42,
                    save_dir=ANALYSIS_CONFIG['results_dir']
                )
                results['trader_classification'] = classification_results
                
                # Store trader features dataframe for later analyses
                if classification_results and 'trader_features' in classification_results:
                    trader_features_df = classification_results['trader_features']
                    print(f"Classified {len(trader_features_df)} traders into groups")
                else:
                    print("Warning: Trader classification didn't produce usable trader features")
            
            # 4. Market Dynamics
            elif stage == "Market Dynamics":
                if 'whale_identification' in results:
                    print("Analyzing market dynamics and whale impact...")
                    whale_ids = results['whale_identification'][0]  # Extract whale IDs from tuple
                    dynamics_results = analyze_market_dynamics(
                        processed_data, 
                        whale_ids, 
                        market_data, 
                        ANALYSIS_CONFIG
                    )
                    results['market_dynamics'] = dynamics_results
                    print(f"Market dynamics analysis completed successfully")
                else:
                    print("Warning: Skipping market dynamics - whale identification results not available")
            
            # Record stage timing
            stage_duration = time.time() - stage_start
            print(f"Completed {stage} in {stage_duration:.1f} seconds")
            
        except Exception as e:
            print(f"Error in {stage}: {e}")
            traceback.print_exc()
    
    # Additional analyses if trader classification was successful
    if ANALYSIS_CONFIG['run_trader_classification'] and trader_features_df is not None:
        print(f"\n{'='*80}\nADDITIONAL ANALYSES\n{'='*80}")
        
        # Temporal Analysis
        try:
            print("\nRunning temporal analysis...")
            start = time.time()
            temporal_results = analyze_trader_temporal_patterns(
                processed_data, 
                trader_features_df
            )
            if temporal_results:
                results['temporal_analysis'] = temporal_results
                print(f"Temporal analysis completed in {time.time() - start:.1f} seconds")
            else:
                print("Warning: Temporal analysis did not produce results")
        except Exception as e:
            print(f"Error in temporal analysis: {e}")
            traceback.print_exc()
        
        # Network Effects
        if 'whale_identification' in results:
            try:
                print("\nRunning network effects analysis...")
                start = time.time()
                network_results = analyze_network_effects(
                    processed_data,
                    trader_features_df,
                    whale_ids,
                    time_window_minutes=60
                )
                if network_results:
                    results['network_effects'] = network_results
                    print(f"Network effects analysis completed in {time.time() - start:.1f} seconds")
                else:
                    print("Warning: Network effects analysis did not produce results")
            except Exception as e:
                print(f"Error in network effects analysis: {e}")
                traceback.print_exc()
        
        # Market Impact
        try:
            print("\nRunning market impact analysis...")
            start = time.time()
            impact_results = analyze_market_impact(
                processed_data, 
                trader_features_df
            )
            if impact_results:
                results['market_impact'] = impact_results
                print(f"Market impact analysis completed in {time.time() - start:.1f} seconds")
            else:
                print("Warning: Market impact analysis did not produce results")
        except Exception as e:
            print(f"Error in market impact analysis: {e}")
            traceback.print_exc()
        
        # Trading Strategy
        try:
            print("\nRunning trading strategy analysis...")
            start = time.time()
            strategy_results = analyze_trading_strategies(
                processed_data, 
                trader_features_df
            )
            if strategy_results:
                results['trading_strategy'] = strategy_results
                print(f"Trading strategy analysis completed in {time.time() - start:.1f} seconds")
            else:
                print("Warning: Trading strategy analysis did not produce results")
        except Exception as e:
            print(f"Error in trading strategy analysis: {e}")
            traceback.print_exc()
    
    # Generate summary report
    # Generate summary report
    if ANALYSIS_CONFIG['save_results']:
        print(f"\n{'='*80}\nGENERATING SUMMARY REPORT\n{'='*80}")
        try:
            generate_summary_report(results, ANALYSIS_CONFIG['results_dir'])
            print("Summary report generated successfully")
        except Exception as e:
            print(f"Error generating summary report: {e}")
            traceback.print_exc()

    # Calculate and report total execution time
    total_duration = time.time() - start_time
    hours, remainder = divmod(total_duration, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"\n{'='*80}")
    print(f"ANALYSIS COMPLETE!")
    print(f"{'='*80}")
    print(f"Total execution time: {int(hours)}h {int(minutes)}m {seconds:.1f}s")
    print(f"Results saved to: {ANALYSIS_CONFIG['results_dir']}")

    return results

In [ ]:
if __name__ == "__main__":
    # Set up the environment
    plt.style.use('seaborn-v0_8-whitegrid')  # For older seaborn
    sns.set_palette("viridis")
    
    # Suppress warnings
    import warnings
    warnings.filterwarnings('ignore')
    
    # Create results directory
    os.makedirs(ANALYSIS_CONFIG['results_dir'], exist_ok=True)
    
    # Run all analyses and collect results
    all_results = run_analysis()
    
    # Print summary
    if all_results:
        print("\nAnalysis completed successfully!")
        
        # Print key metrics if available
        if 'trader_distribution' in all_results and isinstance(all_results['trader_distribution'], tuple):
            dist = all_results['trader_distribution'][0]
            print(f"Total traders analyzed: {dist.get('total_traders', 'N/A'):,}")
        
        if 'whale_identification' in all_results and isinstance(all_results['whale_identification'], tuple):
            whale_ids, whale_results = all_results['whale_identification']
            print(f"Identified {len(whale_ids):,} whale traders")
            print(f"Gini coefficient: {whale_results.get('gini_coefficient', 'N/A'):.4f}")
        
        if 'trader_classification' in all_results and all_results['trader_classification']:
            class_results = all_results['trader_classification']
            if 'cluster_names' in class_results:
                print(f"Identified trader types: {', '.join(class_results['cluster_names'].values())}")